# setup

In [31]:
%%capture
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import json
from fnmatch import fnmatch
from pathlib import Path
from typing import NamedTuple, Optional, Callable, Union, List, Tuple
# from jaxtyping import Float, Int

import einops
import torch
from torch import Tensor, nn
from huggingface_hub import snapshot_download
from natsort import natsorted
from safetensors.torch import load_model, save_model

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from collections import Counter

## corr fns

In [4]:
def batched_correlation(reshaped_activations_A, reshaped_activations_B, batch_size=100):
    # Ensure tensors are on GPU
    if torch.cuda.is_available():
        reshaped_activations_A = reshaped_activations_A.to('cuda')
        reshaped_activations_B = reshaped_activations_B.to('cuda')

    # Normalize columns of A
    mean_A = reshaped_activations_A.mean(dim=0, keepdim=True)
    std_A = reshaped_activations_A.std(dim=0, keepdim=True)
    normalized_A = (reshaped_activations_A - mean_A) / (std_A + 1e-8)  # Avoid division by zero

    # Normalize columns of B
    mean_B = reshaped_activations_B.mean(dim=0, keepdim=True)
    std_B = reshaped_activations_B.std(dim=0, keepdim=True)
    normalized_B = (reshaped_activations_B - mean_B) / (std_B + 1e-8)  # Avoid division by zero

    num_batches = (normalized_B.shape[1] + batch_size - 1) // batch_size
    max_values = []
    max_indices = []

    for batch in range(num_batches):
        start = batch * batch_size
        end = min(start + batch_size, normalized_B.shape[1])
        batch_corr_matrix = torch.matmul(normalized_A.t(), normalized_B[:, start:end]) / normalized_A.shape[0]
        max_val, max_idx = batch_corr_matrix.max(dim=0)
        max_values.append(max_val)
        max_indices.append(max_idx)

        del batch_corr_matrix
        torch.cuda.empty_cache()

    corr_inds = torch.cat(max_indices).detach().cpu().numpy()
    corr_vals = torch.cat(max_values).detach().cpu().numpy()
    return corr_inds, corr_vals

In [5]:
def filter_corr_pairs(mixed_modA_feats, mixed_modB_feats, kept_modA_feats):
    filt_corr_ind_A = []
    filt_corr_ind_B = []
    seen = set()
    for ind_A, ind_B in zip(mixed_modA_feats, mixed_modB_feats):
        if ind_A in kept_modA_feats:
            filt_corr_ind_A.append(ind_A)
            filt_corr_ind_B.append(ind_B)
        elif ind_A not in seen:  # only keep one if it's over count X
            seen.add(ind_A)
            filt_corr_ind_A.append(ind_A)
            filt_corr_ind_B.append(ind_B)
    num_unq_pairs = len(list(set(filt_corr_ind_A)))
    print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
    print("num 1-1 feats after filt: ", len(list(set(filt_corr_ind_A))) )
    return filt_corr_ind_A, filt_corr_ind_B

In [6]:
def get_new_mean_corr(modA_feats, modB_feats, corr_vals):
    new_vals = []
    seen = set()
    for ind_A, ind_B in zip(modA_feats, modB_feats):
        if ind_A not in seen:
            seen.add(ind_A)
            val = corr_vals[ind_B]
            new_vals.append(val)
    new_mean_corr = sum(new_vals) / len(new_vals)
    # print(new_mean_corr)
    return new_mean_corr

## sim fns

In [7]:
import functools
from typing import Any, Callable, Dict, List, Tuple, Union

import numpy as np
import numpy.typing as npt
import torch


def to_numpy_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[npt.NDArray]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> npt.NDArray:
        return x if isinstance(x, np.ndarray) else x.numpy()

    return list(map(convert, args))


def to_torch_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[torch.Tensor]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
        return x if isinstance(x, torch.Tensor) else torch.from_numpy(x)

    return list(map(convert, args))


def adjust_dimensionality(
    R: npt.NDArray, Rp: npt.NDArray, strategy="zero_pad"
) -> Tuple[npt.NDArray, npt.NDArray]:
    D = R.shape[1]
    Dp = Rp.shape[1]
    if strategy == "zero_pad":
        if D - Dp == 0:
            return R, Rp
        elif D - Dp > 0:
            return R, np.concatenate((Rp, np.zeros((Rp.shape[0], D - Dp))), axis=1)
        else:
            return np.concatenate((R, np.zeros((R.shape[0], Dp - D))), axis=1), Rp
    else:
        raise NotImplementedError()


def center_columns(R: npt.NDArray) -> npt.NDArray:
    return R - R.mean(axis=0)[None, :]


def normalize_matrix_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord="fro")


def sim_random_baseline(
    rep1: torch.Tensor, rep2: torch.Tensor, sim_func: Callable, n_permutations: int = 10
) -> Dict[str, Any]:
    torch.manual_seed(1234)
    scores = []
    for _ in range(n_permutations):
        perm = torch.randperm(rep1.size(0))

        score = sim_func(rep1[perm, :], rep2)
        score = score if isinstance(score, float) else score["score"]

        scores.append(score)

    return {"baseline_scores": np.array(scores)}


class Pipeline:
    def __init__(
        self,
        preprocess_funcs: List[Callable[[npt.NDArray], npt.NDArray]],
        similarity_func: Callable[[npt.NDArray, npt.NDArray], Dict[str, Any]],
    ) -> None:
        self.preprocess_funcs = preprocess_funcs
        self.similarity_func = similarity_func

    def __call__(self, R: npt.NDArray, Rp: npt.NDArray) -> Dict[str, Any]:
        for preprocess_func in self.preprocess_funcs:
            R = preprocess_func(R)
            Rp = preprocess_func(Rp)
        return self.similarity_func(R, Rp)

    def __str__(self) -> str:
        def func_name(func: Callable) -> str:
            return (
                func.__name__
                if not isinstance(func, functools.partial)
                else func.func.__name__
            )

        def partial_keywords(func: Callable) -> str:
            if not isinstance(func, functools.partial):
                return ""
            else:
                return str(func.keywords)

        return (
            "Pipeline("
            + (
                "+".join(map(func_name, self.preprocess_funcs))
                + "+"
                + func_name(self.similarity_func)
                + partial_keywords(self.similarity_func)
            )
            + ")"
        )

In [8]:
from typing import List, Set, Union

import numpy as np
import numpy.typing as npt
import sklearn.neighbors
import torch

# from llmcomp.measures.utils import to_numpy_if_needed


def _jac_sim_i(idx_R: Set[int], idx_Rp: Set[int]) -> float:
    return len(idx_R.intersection(idx_Rp)) / len(idx_R.union(idx_Rp))


def jaccard_similarity(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    k: int = 10,
    inner: str = "cosine",
    n_jobs: int = 8,
) -> float:
    R, Rp = to_numpy_if_needed(R, Rp)

    indices_R = nn_array_to_setlist(top_k_neighbors(R, k, inner, n_jobs))
    indices_Rp = nn_array_to_setlist(top_k_neighbors(Rp, k, inner, n_jobs))

    return float(
        np.mean(
            [_jac_sim_i(idx_R, idx_Rp) for idx_R, idx_Rp in zip(indices_R, indices_Rp)]
        )
    )


def top_k_neighbors(
    R: npt.NDArray,
    k: int,
    inner: str,
    n_jobs: int,
) -> npt.NDArray:
    # k+1 nearest neighbors, because we pass in all the data, which means that a point
    # will be the nearest neighbor to itself. We remove this point from the results and
    # report only the k nearest neighbors distinct from the point itself.
    nns = sklearn.neighbors.NearestNeighbors(
        n_neighbors=k + 1, metric=inner, n_jobs=n_jobs
    )
    nns.fit(R)
    _, nns = nns.kneighbors(R)
    return nns[:, 1:]


def nn_array_to_setlist(nn: npt.NDArray) -> List[Set[int]]:
    return [set(idx) for idx in nn]

In [9]:
import functools
import logging
from abc import ABC
from abc import abstractmethod
from dataclasses import dataclass
from dataclasses import field
from typing import Any
from typing import Callable
from typing import get_args
from typing import List
from typing import Literal
from typing import Optional
from typing import Protocol
from typing import Tuple
from typing import Union

import numpy as np
import numpy.typing as npt
import torch
from einops import rearrange
# from loguru import logger

log = logging.getLogger(__name__)


SHAPE_TYPE = Literal["nd", "ntd", "nchw"]

ND_SHAPE, NTD_SHAPE, NCHW_SHAPE = get_args(SHAPE_TYPE)[0], get_args(SHAPE_TYPE)[1], get_args(SHAPE_TYPE)[2]


class SimilarityFunction(Protocol):
    def __call__(  # noqa: E704
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
    ) -> float: ...


class RSMSimilarityFunction(Protocol):
    def __call__(  # noqa: E704
        self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE, n_jobs: int
    ) -> float: ...


@dataclass
class BaseSimilarityMeasure(ABC):
    larger_is_more_similar: bool
    is_symmetric: bool

    is_metric: bool | None = None
    invariant_to_affine: bool | None = None
    invariant_to_invertible_linear: bool | None = None
    invariant_to_ortho: bool | None = None
    invariant_to_permutation: bool | None = None
    invariant_to_isotropic_scaling: bool | None = None
    invariant_to_translation: bool | None = None
    name: str = field(init=False)

    def __post_init__(self):
        self.name = self.__class__.__name__

    @abstractmethod
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        raise NotImplementedError


class FunctionalSimilarityMeasure(BaseSimilarityMeasure):
    @abstractmethod
    def __call__(self, output_a: torch.Tensor | npt.NDArray, output_b: torch.Tensor | npt.NDArray) -> float:
        raise NotImplementedError


@dataclass(kw_only=True)
class RepresentationalSimilarityMeasure(BaseSimilarityMeasure):
    sim_func: SimilarityFunction

    def __call__(
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
    ) -> float:
        return self.sim_func(R, Rp, shape)


class RSMSimilarityMeasure(RepresentationalSimilarityMeasure):
    sim_func: RSMSimilarityFunction

    @staticmethod
    def estimate_good_number_of_jobs(R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray) -> int:
        # RSMs in are NxN (or DxD) so the number of jobs should roughly scale quadratically with increase in N (or D).
        # False! As long as sklearn-native metrics are used, they will use parallel implementations regardless of job
        # count. Each job would spawn their own threads, which leads to oversubscription of cores and thus slowdown.
        # This seems to be not fully correct (n_jobs=2 seems to actually use two cores), but using n_jobs=1 seems the
        # fastest.
        return 1

    def __call__(
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
        n_jobs: Optional[int] = None,
    ) -> float:
        if n_jobs is None:
            n_jobs = self.estimate_good_number_of_jobs(R, Rp)
        return self.sim_func(R, Rp, shape, n_jobs=n_jobs)


def to_numpy_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[npt.NDArray]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> npt.NDArray:
        return x if isinstance(x, np.ndarray) else x.numpy()

    return list(map(convert, args))


def to_torch_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[torch.Tensor]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
        return x if isinstance(x, torch.Tensor) else torch.from_numpy(x)

    return list(map(convert, args))


def adjust_dimensionality(R: npt.NDArray, Rp: npt.NDArray, strategy="zero_pad") -> Tuple[npt.NDArray, npt.NDArray]:
    D = R.shape[1]
    Dp = Rp.shape[1]
    if strategy == "zero_pad":
        if D - Dp == 0:
            return R, Rp
        elif D - Dp > 0:
            return R, np.concatenate((Rp, np.zeros((Rp.shape[0], D - Dp))), axis=1)
        else:
            return np.concatenate((R, np.zeros((R.shape[0], Dp - D))), axis=1), Rp
    else:
        raise NotImplementedError()


def center_columns(R: npt.NDArray) -> npt.NDArray:
    return R - R.mean(axis=0)[None, :]


def normalize_matrix_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord="fro")


def normalize_row_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord=2, axis=1, keepdims=True)


def standardize(R: npt.NDArray) -> npt.NDArray:
    return (R - R.mean(axis=0, keepdims=True)) / R.std(axis=0)


def double_center(x: npt.NDArray) -> npt.NDArray:
    return x - x.mean(axis=0, keepdims=True) - x.mean(axis=1, keepdims=True) + x.mean()


def align_spatial_dimensions(R: npt.NDArray, Rp: npt.NDArray) -> Tuple[npt.NDArray, npt.NDArray]:
    """
    Aligns spatial representations by resizing them to the smallest spatial dimension.
    Subsequent aligned spatial representations are flattened, with the spatial aligned representations
    moving into the *sample* dimension.
    """
    R_re, Rp_re = resize_wh_reps(R, Rp)
    R_re = rearrange(R_re, "n c h w -> (n h w) c")
    Rp_re = rearrange(Rp_re, "n c h w -> (n h w) c")
    if R_re.shape[0] > 5000:
        logger.info(f"Got {R_re.shape[0]} samples in N after flattening. Subsampling to reduce compute.")
        subsample = R_re.shape[0] // 5000
        R_re = R_re[::subsample]
        Rp_re = Rp_re[::subsample]

    return R_re, Rp_re


def average_pool_downsample(R, resize: bool, new_size: tuple[int, int]):
    if not resize:
        return R  # do nothing
    else:
        is_numpy = isinstance(R, np.ndarray)
        R_torch = torch.from_numpy(R) if is_numpy else R
        R_torch = torch.nn.functional.adaptive_avg_pool2d(R_torch, new_size)
        return R_torch.numpy() if is_numpy else R_torch


def resize_wh_reps(R: npt.NDArray, Rp: npt.NDArray) -> Tuple[npt.NDArray, npt.NDArray]:
    """
    Function for resizing spatial representations that are not the same size.
    Does through fourier transform and resizing.

    Args:
        R: numpy array of shape  [batch_size, height, width, num_channels]
        RP: numpy array of shape [batch_size, height, width, num_channels]

    Returns:
        fft_acts1: numpy array of shape [batch_size, (new) height, (new) width, num_channels]
        fft_acts2: numpy array of shape [batch_size, (new) height, (new) width, num_channels]

    """
    height1, width1 = R.shape[2], R.shape[3]
    height2, width2 = Rp.shape[2], Rp.shape[3]
    if height1 != height2 or width1 != width2:
        height = min(height1, height2)
        width = min(width1, width2)
        new_size = [height, width]
        resize = True
    else:
        height = height1
        width = width1
        new_size = None
        resize = False

    # resize and preprocess with fft
    avg_ds1 = average_pool_downsample(R, resize=resize, new_size=new_size)
    avg_ds2 = average_pool_downsample(Rp, resize=resize, new_size=new_size)
    return avg_ds1, avg_ds2


def fft_resize(images, resize=False, new_size=None):
    """Function for applying DFT and resizing.

    This function takes in an array of images, applies the 2-d fourier transform
    and resizes them according to new_size, keeping the frequencies that overlap
    between the two sizes.

    Args:
              images: a numpy array with shape
                      [batch_size, height, width, num_channels]
              resize: boolean, whether or not to resize
              new_size: a tuple (size, size), with height and width the same

    Returns:
              im_fft_downsampled: a numpy array with shape
                           [batch_size, (new) height, (new) width, num_channels]
    """
    assert len(images.shape) == 4, "expecting images to be" "[batch_size, height, width, num_channels]"
    if resize:
        # FFT --> remove high frequencies --> inverse FFT
        im_complex = images.astype("complex64")
        im_fft = np.fft.fft2(im_complex, axes=(1, 2))
        im_shifted = np.fft.fftshift(im_fft, axes=(1, 2))

        center_width = im_shifted.shape[2] // 2
        center_height = im_shifted.shape[1] // 2
        half_w = new_size[0] // 2
        half_h = new_size[1] // 2
        cropped_fft = im_shifted[
            :, center_height - half_h : center_height + half_h, center_width - half_w : center_width + half_w, :
        ]
        cropped_fft_shifted_back = np.fft.ifft2(cropped_fft, axes=(1, 2))
        return cropped_fft_shifted_back.real
    else:
        return images


class Pipeline:
    def __init__(
        self,
        preprocess_funcs: List[Callable[[npt.NDArray], npt.NDArray]],
        similarity_func: Callable[[npt.NDArray, npt.NDArray, SHAPE_TYPE], float],
    ) -> None:
        self.preprocess_funcs = preprocess_funcs
        self.similarity_func = similarity_func

    def __call__(self, R: npt.NDArray, Rp: npt.NDArray, shape: SHAPE_TYPE) -> float:
        try:
            for preprocess_func in self.preprocess_funcs:
                R = preprocess_func(R)
                Rp = preprocess_func(Rp)
            return self.similarity_func(R, Rp, shape)
        except ValueError as e:
            log.info(f"Pipeline failed: {e}")
            return np.nan

    def __str__(self) -> str:
        def func_name(func: Callable) -> str:
            return func.__name__ if not isinstance(func, functools.partial) else func.func.__name__

        def partial_keywords(func: Callable) -> str:
            if not isinstance(func, functools.partial):
                return ""
            else:
                return str(func.keywords)

        return (
            "Pipeline("
            + (
                "+".join(map(func_name, self.preprocess_funcs))
                + "+"
                + func_name(self.similarity_func)
                + partial_keywords(self.similarity_func)
            )
            + ")"
        )


def flatten(*args: Union[torch.Tensor, npt.NDArray], shape: SHAPE_TYPE) -> List[Union[torch.Tensor, npt.NDArray]]:
    if shape == "ntd":
        return list(map(flatten_nxtxd_to_ntxd, args))
    elif shape == "nd":
        return list(args)
    elif shape == "nchw":
        return list(map(flatten_nxcxhxw_to_nxchw, args))  # Flattening non-trivial for nchw
    else:
        raise ValueError("Unknown shape of representations. Must be one of 'ntd', 'nchw', 'nd'.")


def flatten_nxtxd_to_ntxd(R: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
    R = to_torch_if_needed(R)[0]
    log.debug("Shape before flattening: %s", str(R.shape))
    R = torch.flatten(R, start_dim=0, end_dim=1)
    log.debug("Shape after flattening: %s", str(R.shape))
    return R


def flatten_nxcxhxw_to_nxchw(R: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
    R = to_torch_if_needed(R)[0]
    log.debug("Shape before flattening: %s", str(R.shape))
    R = torch.reshape(R, (R.shape[0], -1))
    log.debug("Shape after flattening: %s", str(R.shape))
    return R

In [10]:
import scipy.optimize

def permutation_procrustes(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
    optimal_permutation_alignment: Optional[Tuple[npt.NDArray, npt.NDArray]] = None,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    R, Rp = adjust_dimensionality(R, Rp)

    if not optimal_permutation_alignment:
        PR, PRp = scipy.optimize.linear_sum_assignment(R.T @ Rp, maximize=True)  # returns column assignments
        optimal_permutation_alignment = (PR, PRp)
    PR, PRp = optimal_permutation_alignment
    return float(np.linalg.norm(R[:, PR] - Rp[:, PRp], ord="fro"))

In [11]:
from typing import Optional
from typing import Union

import numpy as np
import numpy.typing as npt
import scipy.spatial.distance
import scipy.stats
import sklearn.metrics
import torch
# from repsim.measures.utils import flatten
# from repsim.measures.utils import RSMSimilarityMeasure
# from repsim.measures.utils import SHAPE_TYPE
# from repsim.measures.utils import to_numpy_if_needed


def representational_similarity_analysis(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
    inner="correlation",
    outer="spearman",
    n_jobs: Optional[int] = None,
) -> float:
    """Representational similarity analysis

    Args:
        R (Union[torch.Tensor, npt.NDArray]): N x D representation
        Rp (Union[torch.Tensor, npt.NDArray]): N x D' representation
        inner (str, optional): inner similarity function for RSM. Must be one of
            scipy.spatial.distance.pdist identifiers . Defaults to "correlation".
        outer (str, optional): outer similarity function that compares RSMs. Defaults to
             "spearman". Must be one of "spearman", "euclidean"

    Returns:
        float: _description_
    """
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)

    if inner == "correlation":
        # n_jobs only works if metric is in PAIRWISE_DISTANCES as defined in sklearn, i.e., not for correlation.
        # But correlation = 1 - cosine dist of row-centered data, so we use the faster cosine metric and center the data.
        R = R - R.mean(axis=1, keepdims=True)
        S = scipy.spatial.distance.squareform(  # take the lower triangle of RSM
            1 - sklearn.metrics.pairwise_distances(R, metric="cosine", n_jobs=n_jobs),  # type:ignore
            checks=False,
        )
        Rp = Rp - Rp.mean(axis=1, keepdims=True)
        Sp = scipy.spatial.distance.squareform(
            1 - sklearn.metrics.pairwise_distances(Rp, metric="cosine", n_jobs=n_jobs),  # type:ignore
            checks=False,
        )
    elif inner == "euclidean":
        # take the lower triangle of RSM
        S = scipy.spatial.distance.squareform(
            sklearn.metrics.pairwise_distances(R, metric=inner, n_jobs=n_jobs), checks=False
        )
        Sp = scipy.spatial.distance.squareform(
            sklearn.metrics.pairwise_distances(Rp, metric=inner, n_jobs=n_jobs), checks=False
        )
    else:
        raise NotImplementedError(f"{inner=}")

    if outer == "spearman":
        return scipy.stats.spearmanr(S, Sp).statistic  # type:ignore
    elif outer == "euclidean":
        return float(np.linalg.norm(S - Sp, ord=2))
    else:
        raise ValueError(f"Unknown outer similarity function: {outer}")


class RSA(RSMSimilarityMeasure):
    def __init__(self):
        # choice of inner/outer in __call__ if fixed to default values, so these values are always the same
        super().__init__(
            sim_func=representational_similarity_analysis,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=True,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=False,
            invariant_to_permutation=True,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

In [12]:
##################################################################################
# Copied from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/cca_core.py
# Copyright 2018 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
The core code for applying Canonical Correlation Analysis to deep networks.

This module contains the core functions to apply canonical correlation analysis
to deep neural networks. The main function is get_cca_similarity, which takes in
two sets of activations, typically the neurons in two layers and their outputs
on all of the datapoints D = [d_1,...,d_m] that have been passed through.

Inputs have shape (num_neurons1, m), (num_neurons2, m). This can be directly
applied used on fully connected networks. For convolutional layers, the 3d block
of neurons can either be flattened entirely, along channels, or alternatively,
the dft_ccas (Discrete Fourier Transform) module can be used.

See:
https://arxiv.org/abs/1706.05806
https://arxiv.org/abs/1806.05759
for full details.

"""
import numpy as np
# from repsim.measures.utils import align_spatial_dimensions

num_cca_trials = 5


def positivedef_matrix_sqrt(array):
    """Stable method for computing matrix square roots, supports complex matrices.

    Args:
              array: A numpy 2d array, can be complex valued that is a positive
                     definite symmetric (or hermitian) matrix

    Returns:
              sqrtarray: The matrix square root of array
    """
    w, v = np.linalg.eigh(array)
    #  A - np.dot(v, np.dot(np.diag(w), v.T))
    wsqrt = np.sqrt(w)
    sqrtarray = np.dot(v, np.dot(np.diag(wsqrt), np.conj(v).T))
    return sqrtarray


def remove_small(sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon):
    """Takes covariance between X, Y, and removes values of small magnitude.

    Args:
              sigma_xx: 2d numpy array, variance matrix for x
              sigma_xy: 2d numpy array, crossvariance matrix for x,y
              sigma_yx: 2d numpy array, crossvariance matrixy for x,y,
                        (conjugate) transpose of sigma_xy
              sigma_yy: 2d numpy array, variance matrix for y
              epsilon : cutoff value for norm below which directions are thrown
                         away

    Returns:
              sigma_xx_crop: 2d array with low x norm directions removed
              sigma_xy_crop: 2d array with low x and y norm directions removed
              sigma_yx_crop: 2d array with low x and y norm directiosn removed
              sigma_yy_crop: 2d array with low y norm directions removed
              x_idxs: indexes of sigma_xx that were removed
              y_idxs: indexes of sigma_yy that were removed
    """

    x_diag = np.abs(np.diagonal(sigma_xx))
    y_diag = np.abs(np.diagonal(sigma_yy))
    x_idxs = x_diag >= epsilon
    y_idxs = y_diag >= epsilon

    sigma_xx_crop = sigma_xx[x_idxs][:, x_idxs]
    sigma_xy_crop = sigma_xy[x_idxs][:, y_idxs]
    sigma_yx_crop = sigma_yx[y_idxs][:, x_idxs]
    sigma_yy_crop = sigma_yy[y_idxs][:, y_idxs]

    return (sigma_xx_crop, sigma_xy_crop, sigma_yx_crop, sigma_yy_crop, x_idxs, y_idxs)


def compute_ccas(sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon, verbose=True):
    """Main cca computation function, takes in variances and crossvariances.

    This function takes in the covariances and cross covariances of X, Y,
    preprocesses them (removing small magnitudes) and outputs the raw results of
    the cca computation, including cca directions in a rotated space, and the
    cca correlation coefficient values.

    Args:
              sigma_xx: 2d numpy array, (num_neurons_x, num_neurons_x)
                        variance matrix for x
              sigma_xy: 2d numpy array, (num_neurons_x, num_neurons_y)
                        crossvariance matrix for x,y
              sigma_yx: 2d numpy array, (num_neurons_y, num_neurons_x)
                        crossvariance matrix for x,y (conj) transpose of sigma_xy
              sigma_yy: 2d numpy array, (num_neurons_y, num_neurons_y)
                        variance matrix for y
              epsilon:  small float to help with stabilizing computations
              verbose:  boolean on whether to print intermediate outputs

    Returns:
              [ux, sx, vx]: [numpy 2d array, numpy 1d array, numpy 2d array]
                            ux and vx are (conj) transposes of each other, being
                            the canonical directions in the X subspace.
                            sx is the set of canonical correlation coefficients-
                            how well corresponding directions in vx, Vy correlate
                            with each other.
              [uy, sy, vy]: Same as above, but for Y space
              invsqrt_xx:   Inverse square root of sigma_xx to transform canonical
                            directions back to original space
              invsqrt_yy:   Same as above but for sigma_yy
              x_idxs:       The indexes of the input sigma_xx that were pruned
                            by remove_small
              y_idxs:       Same as above but for sigma_yy
    """

    (sigma_xx, sigma_xy, sigma_yx, sigma_yy, x_idxs, y_idxs) = remove_small(
        sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon
    )

    numx = sigma_xx.shape[0]
    numy = sigma_yy.shape[0]

    if numx == 0 or numy == 0:
        return (
            [0, 0, 0],
            [0, 0, 0],
            np.zeros_like(sigma_xx),
            np.zeros_like(sigma_yy),
            x_idxs,
            y_idxs,
        )

    if verbose:
        print("adding eps to diagonal and taking inverse")
    sigma_xx += epsilon * np.eye(numx)
    sigma_yy += epsilon * np.eye(numy)
    inv_xx = np.linalg.pinv(sigma_xx)
    inv_yy = np.linalg.pinv(sigma_yy)

    if verbose:
        print("taking square root")
    invsqrt_xx = positivedef_matrix_sqrt(inv_xx)
    invsqrt_yy = positivedef_matrix_sqrt(inv_yy)

    if verbose:
        print("dot products...")
    arr = np.dot(invsqrt_xx, np.dot(sigma_xy, invsqrt_yy))

    if verbose:
        print("trying to take final svd")
    u, s, v = np.linalg.svd(arr)

    if verbose:
        print("computed everything!")

    return [u, np.abs(s), v], invsqrt_xx, invsqrt_yy, x_idxs, y_idxs


def sum_threshold(array, threshold):
    """Computes threshold index of decreasing nonnegative array by summing.

    This function takes in a decreasing array nonnegative floats, and a
    threshold between 0 and 1. It returns the index i at which the sum of the
    array up to i is threshold*total mass of the array.

    Args:
              array: a 1d numpy array of decreasing, nonnegative floats
              threshold: a number between 0 and 1

    Returns:
              i: index at which np.sum(array[:i]) >= threshold
    """
    assert (threshold >= 0) and (threshold <= 1), "print incorrect threshold"

    for i in range(len(array)):
        if np.sum(array[:i]) / np.sum(array) >= threshold:
            return i


def create_zero_dict(compute_dirns, dimension):
    """Outputs a zero dict when neuron activation norms too small.

    This function creates a return_dict with appropriately shaped zero entries
    when all neuron activations are very small.

    Args:
              compute_dirns: boolean, whether to have zero vectors for directions
              dimension: int, defines shape of directions

    Returns:
              return_dict: a dict of appropriately shaped zero entries
    """
    return_dict = {}
    return_dict["mean"] = (np.asarray(0), np.asarray(0))
    return_dict["sum"] = (np.asarray(0), np.asarray(0))
    return_dict["cca_coef1"] = np.asarray(0)
    return_dict["cca_coef2"] = np.asarray(0)
    return_dict["idx1"] = 0
    return_dict["idx2"] = 0

    if compute_dirns:
        return_dict["cca_dirns1"] = np.zeros((1, dimension))
        return_dict["cca_dirns2"] = np.zeros((1, dimension))

    return return_dict


def get_cca_similarity(
    acts1,
    acts2,
    epsilon=0.0,
    threshold=0.98,
    compute_coefs=True,
    compute_dirns=False,
    verbose=True,
):
    """The main function for computing cca similarities.

    This function computes the cca similarity between two sets of activations,
    returning a dict with the cca coefficients, a few statistics of the cca
    coefficients, and (optionally) the actual directions.

    Args:
              acts1: (num_neurons1, data_points) a 2d numpy array of neurons by
                     datapoints where entry (i,j) is the output of neuron i on
                     datapoint j.
              acts2: (num_neurons2, data_points) same as above, but (potentially)
                     for a different set of neurons. Note that acts1 and acts2
                     can have different numbers of neurons, but must agree on the
                     number of datapoints

              epsilon: small float to help stabilize computations

              threshold: float between 0, 1 used to get rid of trailing zeros in
                         the cca correlation coefficients to output more accurate
                         summary statistics of correlations.


              compute_coefs: boolean value determining whether coefficients
                             over neurons are computed. Needed for computing
                             directions

              compute_dirns: boolean value determining whether actual cca
                             directions are computed. (For very large neurons and
                             datasets, may be better to compute these on the fly
                             instead of store in memory.)

              verbose: Boolean, whether intermediate outputs are printed

    Returns:
              return_dict: A dictionary with outputs from the cca computations.
                           Contains neuron coefficients (combinations of neurons
                           that correspond to cca directions), the cca correlation
                           coefficients (how well aligned directions correlate),
                           x and y idxs (for computing cca directions on the fly
                           if compute_dirns=False), and summary statistics. If
                           compute_dirns=True, the cca directions are also
                           computed.
    """

    # assert dimensionality equal
    assert acts1.shape[1] == acts2.shape[1], "dimensions don't match"
    # check that acts1, acts2 are transposition
    assert acts1.shape[0] < acts1.shape[1], "input must be number of neurons" "by datapoints"
    return_dict = {}

    # compute covariance with numpy function for extra stability
    numx = acts1.shape[0]
    numy = acts2.shape[0]

    covariance = np.cov(acts1, acts2)
    sigmaxx = covariance[:numx, :numx]
    sigmaxy = covariance[:numx, numx:]
    sigmayx = covariance[numx:, :numx]
    sigmayy = covariance[numx:, numx:]

    # rescale covariance to make cca computation more stable
    xmax = np.max(np.abs(sigmaxx))
    ymax = np.max(np.abs(sigmayy))
    sigmaxx /= xmax
    sigmayy /= ymax
    sigmaxy /= np.sqrt(xmax * ymax)
    sigmayx /= np.sqrt(xmax * ymax)

    ([u, s, v], invsqrt_xx, invsqrt_yy, x_idxs, y_idxs) = compute_ccas(
        sigmaxx, sigmaxy, sigmayx, sigmayy, epsilon=epsilon, verbose=verbose
    )

    # if x_idxs or y_idxs is all false, return_dict has zero entries
    if (not np.any(x_idxs)) or (not np.any(y_idxs)):
        return create_zero_dict(compute_dirns, acts1.shape[1])

    if compute_coefs:
        # also compute full coefficients over all neurons
        x_mask = np.dot(x_idxs.reshape((-1, 1)), x_idxs.reshape((1, -1)))
        y_mask = np.dot(y_idxs.reshape((-1, 1)), y_idxs.reshape((1, -1)))

        return_dict["coef_x"] = u.T
        return_dict["invsqrt_xx"] = invsqrt_xx
        return_dict["full_coef_x"] = np.zeros((numx, numx))
        np.place(return_dict["full_coef_x"], x_mask, return_dict["coef_x"])
        return_dict["full_invsqrt_xx"] = np.zeros((numx, numx))
        np.place(return_dict["full_invsqrt_xx"], x_mask, return_dict["invsqrt_xx"])

        return_dict["coef_y"] = v
        return_dict["invsqrt_yy"] = invsqrt_yy
        return_dict["full_coef_y"] = np.zeros((numy, numy))
        np.place(return_dict["full_coef_y"], y_mask, return_dict["coef_y"])
        return_dict["full_invsqrt_yy"] = np.zeros((numy, numy))
        np.place(return_dict["full_invsqrt_yy"], y_mask, return_dict["invsqrt_yy"])

        # compute means
        neuron_means1 = np.mean(acts1, axis=1, keepdims=True)
        neuron_means2 = np.mean(acts2, axis=1, keepdims=True)
        return_dict["neuron_means1"] = neuron_means1
        return_dict["neuron_means2"] = neuron_means2

    if compute_dirns:
        # orthonormal directions that are CCA directions
        cca_dirns1 = (
            np.dot(
                np.dot(return_dict["full_coef_x"], return_dict["full_invsqrt_xx"]),
                (acts1 - neuron_means1),
            )
            + neuron_means1
        )
        cca_dirns2 = (
            np.dot(
                np.dot(return_dict["full_coef_y"], return_dict["full_invsqrt_yy"]),
                (acts2 - neuron_means2),
            )
            + neuron_means2
        )

    # get rid of trailing zeros in the cca coefficients
    idx1 = sum_threshold(s, threshold)
    idx2 = sum_threshold(s, threshold)

    return_dict["cca_coef1"] = s
    return_dict["cca_coef2"] = s
    return_dict["x_idxs"] = x_idxs
    return_dict["y_idxs"] = y_idxs
    # summary statistics
    return_dict["mean"] = (np.mean(s[:idx1]), np.mean(s[:idx2]))
    return_dict["sum"] = (np.sum(s), np.sum(s))

    if compute_dirns:
        return_dict["cca_dirns1"] = cca_dirns1
        return_dict["cca_dirns2"] = cca_dirns2

    return return_dict


def robust_cca_similarity(acts1, acts2, threshold=0.98, epsilon=1e-6, compute_dirns=True):
    """Calls get_cca_similarity multiple times while adding noise.

    This function is very similar to get_cca_similarity, and can be used if
    get_cca_similarity doesn't converge for some pair of inputs. This function
    adds some noise to the activations to help convergence.

    Args:
              acts1: (num_neurons1, data_points) a 2d numpy array of neurons by
                     datapoints where entry (i,j) is the output of neuron i on
                     datapoint j.
              acts2: (num_neurons2, data_points) same as above, but (potentially)
                     for a different set of neurons. Note that acts1 and acts2
                     can have different numbers of neurons, but must agree on the
                     number of datapoints

              threshold: float between 0, 1 used to get rid of trailing zeros in
                         the cca correlation coefficients to output more accurate
                         summary statistics of correlations.

              epsilon: small float to help stabilize computations

              compute_dirns: boolean value determining whether actual cca
                             directions are computed. (For very large neurons and
                             datasets, may be better to compute these on the fly
                             instead of store in memory.)

    Returns:
              return_dict: A dictionary with outputs from the cca computations.
                           Contains neuron coefficients (combinations of neurons
                           that correspond to cca directions), the cca correlation
                           coefficients (how well aligned directions correlate),
                           x and y idxs (for computing cca directions on the fly
                           if compute_dirns=False), and summary statistics. If
                           compute_dirns=True, the cca directions are also
                           computed.
    """

    for trial in range(num_cca_trials):
        try:
            return_dict = get_cca_similarity(acts1, acts2, threshold, compute_dirns)
        except np.linalg.LinAlgError:
            acts1 = acts1 * 1e-1 + np.random.normal(size=acts1.shape) * epsilon
            acts2 = acts2 * 1e-1 + np.random.normal(size=acts1.shape) * epsilon
            if trial + 1 == num_cca_trials:
                raise

    return return_dict
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/cca_core.py


def top_k_pca_comps(singular_values, threshold=0.99):
    total_variance = np.sum(singular_values**2)
    explained_variance = (singular_values**2) / total_variance
    cumulative_variance = np.cumsum(explained_variance)
    return np.argmax(cumulative_variance >= threshold * total_variance) + 1


def _svcca_original(acts1, acts2):
    # Copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/tutorials/001_Introduction.ipynb
    # Modification: get_cca_similarity is in the same file.
    # Modification: top-k PCA component selection s.t. explained variance > 0.99 total variance
    # Mean subtract activations
    cacts1 = acts1 - np.mean(acts1, axis=1, keepdims=True)
    cacts2 = acts2 - np.mean(acts2, axis=1, keepdims=True)

    # Perform SVD
    U1, s1, V1 = np.linalg.svd(cacts1, full_matrices=False)
    U2, s2, V2 = np.linalg.svd(cacts2, full_matrices=False)

    # top-k PCA components only
    k1 = top_k_pca_comps(s1)
    k2 = top_k_pca_comps(s2)

    svacts1 = np.dot(s1[:k1] * np.eye(k1), V1[:k1])
    # can also compute as svacts1 = np.dot(U1.T[:20], cacts1)
    svacts2 = np.dot(s2[:k2] * np.eye(k2), V2[:k2])
    # can also compute as svacts1 = np.dot(U2.T[:20], cacts2)

    svcca_results = get_cca_similarity(svacts1, svacts2, epsilon=1e-10, verbose=False)
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/tutorials/001_Introduction.ipynb
    return np.mean(svcca_results["cca_coef1"])


# Copied from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/pwcca.py
# Modification: get_cca_similarity is in the same file.
def compute_pwcca(acts1, acts2, epsilon=0.0):
    """Computes projection weighting for weighting CCA coefficients

    Args:
         acts1: 2d numpy array, shaped (neurons, num_datapoints)
         acts2: 2d numpy array, shaped (neurons, num_datapoints)

    Returns:
         Original cca coefficient mean and weighted mean

    """
    sresults = get_cca_similarity(
        acts1,
        acts2,
        epsilon=epsilon,
        compute_dirns=False,
        compute_coefs=True,
        verbose=False,
    )
    if np.sum(sresults["x_idxs"]) <= np.sum(sresults["y_idxs"]):
        dirns = (
            np.dot(
                sresults["coef_x"],
                (acts1[sresults["x_idxs"]] - sresults["neuron_means1"][sresults["x_idxs"]]),
            )
            + sresults["neuron_means1"][sresults["x_idxs"]]
        )
        coefs = sresults["cca_coef1"]
        acts = acts1
        idxs = sresults["x_idxs"]
    else:
        dirns = (
            np.dot(
                sresults["coef_y"],
                (acts1[sresults["y_idxs"]] - sresults["neuron_means2"][sresults["y_idxs"]]),
            )
            + sresults["neuron_means2"][sresults["y_idxs"]]
        )
        coefs = sresults["cca_coef2"]
        acts = acts2
        idxs = sresults["y_idxs"]
    P, _ = np.linalg.qr(dirns.T)
    weights = np.sum(np.abs(np.dot(P.T, acts[idxs].T)), axis=1)
    weights = weights / np.sum(weights)

    return np.sum(weights * coefs), weights, coefs
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/pwcca.py


##################################################################################

from typing import Union  # noqa:e402

import numpy.typing as npt  # noqa:e402
import torch  # noqa:e402

# from repsim.measures.utils import (
#     SHAPE_TYPE,
#     flatten,
#     resize_wh_reps,
#     to_numpy_if_needed,
#     RepresentationalSimilarityMeasure,
# )  # noqa:e402


def svcca(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    return _svcca_original(R.T, Rp.T)


def pwcca(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    return compute_pwcca(R.T, Rp.T)[0]


class SVCCA(RepresentationalSimilarityMeasure):
    def __init__(self):
        super().__init__(
            sim_func=svcca,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=True,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=True,
            invariant_to_permutation=True,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

    def __call__(self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE) -> float:
        if shape == "nchw":
            # Move spatial dimensions into the sample dimension
            # If not the same spatial dimension, resample via FFT.
            R, Rp = align_spatial_dimensions(R, Rp)
            shape = "nd"

        return self.sim_func(R, Rp, shape)


class PWCCA(RepresentationalSimilarityMeasure):
    def __init__(self):
        super().__init__(
            sim_func=pwcca,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=False,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=False,
            invariant_to_permutation=False,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

    def __call__(self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE) -> float:
        if shape == "nchw":
            # Move spatial dimensions into the sample dimension
            # If not the same spatial dimension, resample via FFT.
            R, Rp = align_spatial_dimensions(R, Rp)
            shape = "nd"

        return self.sim_func(R, Rp, shape)

## get rand

In [13]:
def score_rand(num_runs, weight_matrix_np, weight_matrix_2, num_feats, sim_fn, shapereq_bool):
    all_rand_scores = []
    i = 0
    # for i in range(num_runs):
    while i < num_runs:
        try:
            rand_modA_feats = np.random.choice(range(weight_matrix_np.shape[0]), size=num_feats, replace=False).tolist()
            rand_modB_feats = np.random.choice(range(weight_matrix_2.shape[0]), size=num_feats, replace=False).tolist()

            if shapereq_bool:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats], "nd")
            else:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats])
            all_rand_scores.append(score)
            i += 1
        except:
            continue
    return sum(all_rand_scores) / len(all_rand_scores)

In [14]:
def score_rand_corr(num_runs, weight_matrix_np, weight_matrix_2, num_feats, highest_correlations_indices_AB, sim_fn, shapereq_bool):
    all_rand_scores = []
    i = 0
    # for i in range(num_runs):
    while i < num_runs:
        try:
            rand_modB_feats = np.random.choice(range(weight_matrix_2.shape[0]), size=num_feats, replace=False).tolist()
            rand_modA_feats = [highest_correlations_indices_AB[index] for index in rand_modB_feats]

            if shapereq_bool:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats], "nd")
            else:
                score = sim_fn(weight_matrix_np[rand_modA_feats], weight_matrix_2[rand_modB_feats])
            all_rand_scores.append(score)
            i += 1
        except:
            continue
    # print(sum(all_rand_scores) / len(all_rand_scores))
    # plt.hist(all_rand_scores)
    # plt.show()
    return all_rand_scores

In [15]:
import random
def shuffle_rand(num_runs, weight_matrix_np, weight_matrix_2, num_feats, sim_fn, shapereq_bool):
    all_rand_scores = []
    for i in range(num_runs):
        row_idxs = list(range(num_feats))
        random.shuffle(row_idxs)
        if shapereq_bool:
            score = sim_fn(weight_matrix_np, weight_matrix_2[row_idxs], "nd")
        else:
            score = sim_fn(weight_matrix_np, weight_matrix_2[row_idxs])
        all_rand_scores.append(score)
    # return sum(all_rand_scores) / len(all_rand_scores)
    return all_rand_scores

## plot fns

In [16]:
def plot_svcca_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['svcca_paired'] for i in range(0, 12)]
    unpaired_values = [layer_to_dictscores[i]['svcca_unpaired'] for i in range(0, 12)]

    # Plotting configuration
    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    # Increase figure size
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
    rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

    # Adding labels, title and custom x-axis tick labels
    ax.set_ylabel('SVCCA')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    # Rotate labels and adjust padding
    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            # if i % 2 == 0:  # Label every other bar above
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12) # , rotation=90
            # else:  # Label every other bar below
            #     ax.text(rect.get_x() + rect.get_width()/2., 0,
            #             f'{height:.3f}',
            #             ha='center', va='top', rotation=90)

    label_bars(rects1)
    label_bars(rects2)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    plt.show()

In [17]:
def plot_rsa_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['rsa_paired'] for i in range(0, 12)]
    unpaired_values = [layer_to_dictscores[i]['rsa_unpaired'] for i in range(0, 12)]

    # Plotting configuration
    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    # Increase figure size
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
    rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

    # Adding labels, title and custom x-axis tick labels
    ax.set_ylabel('RSA')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    # Rotate labels and adjust padding
    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            # if i % 2 == 0:  # Label every other bar above
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12) # , rotation=90
            # else:  # Label every other bar below
            #     ax.text(rect.get_x() + rect.get_width()/2., 0,
            #             f'{height:.3f}',
            #             ha='center', va='top', rotation=90)

    label_bars(rects1)
    label_bars(rects2)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    plt.show()

In [18]:
def plot_meanCorr_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['mean_actv_corr'] for i in range(0, 12)]
    # unpaired_values = [layer_to_dictscores[i]['svcca_unpaired'] for i in range(0, 12)]

    # Plotting configuration
    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    # Increase figure size
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
    # rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

    # Adding labels, title and custom x-axis tick labels
    ax.set_ylabel('Corr')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    # Rotate labels and adjust padding
    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            # if i % 2 == 0:  # Label every other bar above
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12) # , rotation=90
            # else:  # Label every other bar below
            #     ax.text(rect.get_x() + rect.get_width()/2., 0,
            #             f'{height:.3f}',
            #             ha='center', va='top', rotation=90)

    label_bars(rects1)
    # label_bars(rects2)

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    plt.show()

In [19]:
def plot_meanCorr_filt_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['mean_actv_corr_filt'] for i in range(0, 12)]

    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')

    ax.set_ylabel('Corr')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12)

    label_bars(rects1)
    plt.tight_layout()

    plt.show()

In [20]:
def plot_numFeats_afterFilt_byLayer(layer_to_dictscores):
    for key, sub_dict in layer_to_dictscores.items():
        for sub_key, value in sub_dict.items():
            sub_dict[sub_key] = round(value, 4)

    layers = [f'L{i}' for i in range(0, 12)]
    paired_values = [layer_to_dictscores[i]['num_feat_filt'] for i in range(0, 12)]

    x = np.arange(len(layers))  # label locations
    width = 0.35  # width of the bars

    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')

    ax.set_ylabel('Num Feats Kept')
    ax.set_title(f'SAEs comparison by Pythia 70m MLP{layer_id} vs 160m MLP Layers')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
    ax.legend()

    def label_bars(rects):
        for i, rect in enumerate(rects):
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom', fontsize=12)

    label_bars(rects1)
    plt.tight_layout()

    plt.show()

In [21]:
# def plot_js_byLayer(layer_to_dictscores):
#     for key, sub_dict in layer_to_dictscores.items():
#         for sub_key, value in sub_dict.items():
#             sub_dict[sub_key] = round(value, 4)

#     layers = [f'L{i}' for i in range(0, 12)]
#     paired_values = [layer_to_dictscores[i]['1-1 jaccard_paired'] for i in range(0, 12)]
#     unpaired_values = [layer_to_dictscores[i]['1-1 jaccard_unpaired'] for i in range(0, 12)]

#     # Plotting configuration
#     x = np.arange(len(layers))  # label locations
#     width = 0.35  # width of the bars

#     # Increase figure size
#     fig, ax = plt.subplots(figsize=(12, 7))  # Slightly increased figure size
#     rects1 = ax.bar(x - width/2, paired_values, width, label='Paired')
#     rects2 = ax.bar(x + width/2, unpaired_values, width, label='Unpaired')

#     # Adding labels, title and custom x-axis tick labels
#     ax.set_ylabel('Jaccard NN')
#     ax.set_title('SAEs comparison by Pythia 70m MLP0 vs 160m MLP Layers')
#     ax.set_xticks(x)
#     ax.set_xticklabels(layers)
#     ax.set_ylim(0, 1)  # Ensuring y-axis is scaled from 0 to 1
#     ax.legend()

#     # Label bars with increased font size and different positioning for paired and unpaired
#     def label_bars(rects, is_paired):
#         for rect in rects:
#             height = rect.get_height()
#             label_height = height + 0.05 if is_paired else height + 0.01
#             ax.text(rect.get_x() + rect.get_width()/2., label_height,
#                     f'{height:.3f}',
#                     ha='center', va='bottom', fontsize=9)  # Increased font size to 12

#     label_bars(rects1, True)   # Paired bars
#     label_bars(rects2, False)  # Unpaired bars

#     # Adjust layout to prevent cutting off labels
#     plt.tight_layout()

#     # Increase y-axis limit to accommodate higher labels
#     ax.set_ylim(0, 1.1)  # Increased from 1 to 1.1

#     plt.show()

## interpret fns

In [22]:
def highest_activating_tokens(
    feature_acts,
    feature_idx: int,
    k: int = 10,  # num batch_seq samples
    batch_tokens=None
): # -> Tuple[Int[Tensor, "k 2"], Float[Tensor, "k"]]:
    '''
    Returns the indices & values for the highest-activating tokens in the given batch of data.
    '''
    batch_size, seq_len = batch_tokens.shape

    # Get the top k largest activations for only targeted feature
    # need to flatten (batch,seq) into batch*seq first because it's ANY batch_seq, even if in same batch or same pos
    flattened_feature_acts = feature_acts[:, :, feature_idx].reshape(-1)

    top_acts_values, top_acts_indices = flattened_feature_acts.topk(k)
    # top_acts_values should be 1D
    # top_acts_indices should be also be 1D. Now, turn it back to 2D
    # Convert the indices into (batch, seq) indices
    top_acts_batch = top_acts_indices // seq_len
    top_acts_seq = top_acts_indices % seq_len

    return torch.stack([top_acts_batch, top_acts_seq], dim=-1), top_acts_values

In [23]:
from rich import print as rprint
def display_top_sequences(top_acts_indices, top_acts_values, batch_tokens):
    s = ""
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        # s += f'{batch_idx}\n'
        s += f'batchID: {batch_idx}, '
        # Get the sequence as a string (with some padding on either side of our sequence)
        seq_start = max(seq_idx - 5, 0)
        seq_end = min(seq_idx + 5, batch_tokens.shape[1])
        seq = ""
        # Loop over the sequence, adding each token to the string (highlighting the token with the large activations)
        for i in range(seq_start, seq_end):
            # new_str_token = model.to_single_str_token(batch_tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            new_str_token = tokenizer.decode([batch_tokens[batch_idx, i].item()]).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            if i == seq_idx:
                new_str_token = f"[bold u dark_orange]{new_str_token}[/]"
            seq += new_str_token
        # Print the sequence, and the activation value
        s += f'Act = {value:.2f}, Seq = "{seq}"\n'

    rprint(s)

## search modB, get modA pair

In [24]:
def find_indices_with_keyword(fList, keyword):
    """
    Find all indices of fList which contain the keyword in the string at those indices.

    Args:
    fList (list of str): List of strings to search within.
    keyword (str): Keyword to search for within the strings of fList.

    Returns:
    list of int: List of indices where the keyword is found within the strings of fList.
    """
    index_list = []
    for index, split_list in enumerate(fList):
        no_space_list = [i.replace(' ', '').lower() for i in split_list]
        if keyword in no_space_list:
            index_list.append(index)
    return index_list

## get concept space features

In [49]:
# def get_mixed_feats(keywords):
def get_mixed_feats(fList_model_B, corr_inds, keywords):
    mixed_modA_feats = []
    mixed_modB_feats = []
    for kw in keywords:
        modB_feats = find_indices_with_keyword(fList_model_B, kw)
        modA_feats = [corr_inds[index] for index in modB_feats]
        mixed_modA_feats.extend(modA_feats)
        mixed_modB_feats.extend(modB_feats)

    print("Unique modA feats: ", len(list(set(mixed_modA_feats))) )
    print("Unique modB feats: ",  len(list(set(mixed_modB_feats))) )
    return mixed_modA_feats, mixed_modB_feats

## get actv fns

In [27]:
def count_zero_columns(tensor):
    # Check if all elements in each column are zero
    zero_columns = np.all(tensor == 0, axis=0)
    # Count True values in the zero_columns array
    zero_cols_indices = np.where(zero_columns)[0]
    return np.sum(zero_columns), zero_cols_indices

## run expm fns

In [28]:
def run_expm(layer_id, outputs, outputs_2):
    layer_to_dictscores = {}

    name = "EleutherAI/sae-pythia-70m-32k"
    cfg_dict = {"expansion_factor": 32, "normalize_decoder": True, "num_latents": 32768, "k": 16, "d_in": 512}
    weight_matrix_np, reshaped_activations_A = get_weights_and_acts(name, cfg_dict, layer_id, outputs)
    # zero_cols_count, zero_cols_indices = count_zero_columns(reshaped_activations_A.cpu().numpy())
    # print("Number of zero columns:", zero_cols_count) #, zero_cols_indices

    name = "EleutherAI/sae-pythia-160m-32k"
    for layerID_2 in range(0, 12): # 0, 12
        dictscores = {}

        # redef
        cfg_dict = {"expansion_factor": 32, "normalize_decoder": True, "num_latents": 32768, "k": 16, "d_in": 768}
        weight_matrix_2, reshaped_activations_B = get_weights_and_acts(name, cfg_dict, layerID_2, outputs_2)


        highest_correlations_indices_AB, highest_correlations_values_AB = batched_correlation(reshaped_activations_A, reshaped_activations_B)

        num_unq_pairs = len(list(set(highest_correlations_indices_AB)))
        print("% unique: ", num_unq_pairs / len(highest_correlations_indices_AB))

        dictscores["mean_actv_corr"] = sum(highest_correlations_values_AB) / len(highest_correlations_values_AB)

        ###########
        # filter

        sorted_feat_counts = Counter(highest_correlations_indices_AB).most_common()
        # kept_modA_feats = [feat_ID for feat_ID, count in sorted_feat_counts if count <= 100]
        kept_modA_feats = [feat_ID for feat_ID, count in sorted_feat_counts if count <= 1]

        filt_corr_ind_A = []
        filt_corr_ind_B = []
        seen = set()
        for ind_B, ind_A in enumerate(highest_correlations_indices_AB):
            if ind_A in kept_modA_feats:
                filt_corr_ind_A.append(ind_A)
                filt_corr_ind_B.append(ind_B)
            elif ind_A not in seen:  # only keep one if it's over count X
                seen.add(ind_A)
                filt_corr_ind_A.append(ind_A)
                filt_corr_ind_B.append(ind_B)
        # num_unq_pairs = len(list(set(filt_corr_ind_A)))
        # print("% unique: ", num_unq_pairs / len(filt_corr_ind_A))
        # print("num feats after filt: ", len(filt_corr_ind_A))

        new_highest_correlations_indices_A = []
        new_highest_correlations_indices_B = []
        new_highest_correlations_values = []

        for ind_A, ind_B in zip(filt_corr_ind_A, filt_corr_ind_B):
            val = highest_correlations_values_AB[ind_B]
            if val > 0:
                new_highest_correlations_indices_A.append(ind_A)
                new_highest_correlations_indices_B.append(ind_B)
                new_highest_correlations_values.append(val)

        # print("% unique after rmv 0s: ", num_unq_pairs / len(new_highest_correlations_indices_A))
        # print("num feats after rmv 0s: ", len(new_highest_correlations_indices_A))
        dictscores["num_feat_kept"] = len(new_highest_correlations_indices_A)

        dictscores["mean_actv_corr_filt"] = sum(new_highest_correlations_values) / len(new_highest_correlations_values)

        ###########
        # sim tests

        # # num_feats = len(filt_corr_ind_A)
        num_feats = len(new_highest_correlations_indices_A)

        # # dictscores["svcca_paired"] = svcca(weight_matrix_np[filt_corr_ind_A], weight_matrix_2[filt_corr_ind_B], "nd")
        # dictscores["svcca_paired"] = svcca(weight_matrix_np[new_highest_correlations_indices_A], weight_matrix_2[new_highest_correlations_indices_B], "nd")

        # dictscores["svcca_unpaired"] = score_rand(weight_matrix_np, weight_matrix_2, num_feats,
        #                                           svcca, shapereq_bool=True)

        dictscores["rsa_paired"] = representational_similarity_analysis(weight_matrix_np[new_highest_correlations_indices_A], weight_matrix_2[new_highest_correlations_indices_B], "nd")
        dictscores["rsa_unpaired"] = score_rand(weight_matrix_np, weight_matrix_2, num_feats,
                                                  representational_similarity_analysis, shapereq_bool=True)

        print("Layer: " + str(layerID_2))
        for key, value in dictscores.items():
            print(key + ": " + str(value))
        print("\n")

        layer_to_dictscores[layerID_2] = dictscores
    return layer_to_dictscores

# load data

In [32]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [33]:
from datasets import load_dataset
# dataset = load_dataset("roneneldan/TinyStories", split="train", streaming=True)
dataset = load_dataset("Skylion007/openwebtext", split="train", streaming=True)

openwebtext.py:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.35k [00:00<?, ?B/s]

The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [34]:
batch_size = 100
maxseqlen = 300

def get_next_batch(dataset_iter, batch_size=100):
    batch = []
    for _ in range(batch_size):
        try:
            sample = next(dataset_iter)
            batch.append(sample['text'])
        except StopIteration:
            break
    return batch

dataset_iter = iter(dataset)
batch = get_next_batch(dataset_iter, batch_size)

# Tokenize the batch
inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=maxseqlen)

# load models

In [35]:
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")
model_2 = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-160m")

config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

## get LLM actvs

In [36]:
with torch.inference_mode():
    outputs = model(**inputs, output_hidden_states=True)
    outputs_2 = model_2(**inputs, output_hidden_states=True)

# concept keywords

In [37]:
keywords = {}

keywords['numeric'] = [
    "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",
    "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen", "twenty",
    "hundred", "thousand", "million", "billion", "trillion",
    "integer", "fraction", "decimal", "percentage", "ratio",
    "numeral", "digit", "prime", "even", "odd",
    "sum", "difference", "product", "factor", "multiple",
    "total", "count", "measure", "dozen", "score", "unit"]

keywords['people'] = [
    "man", "girl", "boy", "kid", "dad", "mom", "son", "sis", "bro",
    "pal", "mate", "boss", "chief", "cop", "guide", "priest", "king",
    "queen", "duke", "lord", "friend", "judge", "clerk", "coach", "team",
    "crew", "staff", "nurse", "doc", "vet", "cook", "maid", "clown",
    "star", "clan", "host", "guest", "peer", "guard", "boss", "spy",
    "fool", "punk", "nerd", "jock", "chief", "folk", "crowd"]

keywords['nature'] = [
    "tree", "grass", "bush", "plant", "stone", "rock", "cliff", "hill",
    "dirt", "sand", "mud", "wind", "storm", "rain", "cloud", "sun",
    "moon", "star", "leaf", "branch", "twig", "root", "bark", "seed",
    "wave", "tide", "lake", "pond", "creek", "sea", "wood", "field",
    "shore", "snow", "ice", "flame", "fire", "fog", "dew", "hail",
    "sky", "earth", "glade", "cave", "peak", "ridge", "dust", "air",
    "mist", "heat"]

keywords['science'] = [
    "cell", "gene", "nerve", "brain", "blood", "bone", "heart", "lung",
    "star", "space", "light", "mass", "force", "wave", "speed", "sound",
    "time", "power", "heat", "cold", "charge", "spark", "flame", "bond",
    "quark", "atom", "ion", "gas", "wind", "ice", "plant", "rock",
    "probe", "test", "fact", "proof", "code", "law", "rule", "graph",
    "scale", "scope", "lens", "ray", "line", "chart", "flux", "phase",
    "shock", "pulse"]

keywords['animals'] = [
    "dog", "cat", "rat", "bat", "pig", "cow", "fox", "wolf", "ram", "eel",
    "ant", "bee", "bug", "cub", "kit", "fawn", "calf", "colt", "foal",
    "hen", "duck", "goat", "bird", "crow", "fish", "frog", "deer", "worm",
    "moth", "gnat", "clam", "crab", "shrimp", "whale", "shark", "squid",
    "pup", "joey", "owl", "hare", "seal", "mule", "toad", "swan", "sow",
    "bull", "stag", "buck", "boar", "kite"
]

# MLP 3 vs L5

## get model actvs

In [38]:
mod_A_layer = 3

weight_matrix_np = model.gpt_neox.layers[mod_A_layer].mlp.dense_4h_to_h.weight
weight_matrix_np = weight_matrix_np.cpu().detach().numpy()

# with torch.inference_mode():
#     outputs = model(**inputs, output_hidden_states=True)
hidden_state = outputs.hidden_states[mod_A_layer].to("cuda")

first_dim_reshaped = hidden_state.shape[0] * hidden_state.shape[1]
reshaped_activations_A = hidden_state.reshape(first_dim_reshaped, hidden_state.shape[-1]).cpu()

del hidden_state
torch.cuda.empty_cache()

In [40]:
mod_B_layer = 5

weight_matrix_2 = model_2.gpt_neox.layers[mod_B_layer].mlp.dense_4h_to_h.weight
weight_matrix_2 = weight_matrix_2.cpu().detach().numpy()

with torch.inference_mode():
    hidden_state_2 = outputs_2.hidden_states[mod_B_layer].to("cuda")

reshaped_activations_B = hidden_state_2.reshape(first_dim_reshaped, hidden_state_2.shape[-1]).cpu()
del hidden_state_2
torch.cuda.empty_cache()

## corr

In [41]:
"""
`batched_correlation(reshaped_activations_B, reshaped_activations_A)` : highest_correlations_indices_AB contains modA's feats as inds, and modB's feats as vals. Use the list with smaller number of features (cols) as the second arg
(reshaped_activations_A, reshaped_activations_B): modB is inds, modA is vals of highest_correlations_indices_AB
"""
corr_inds, corr_vals = batched_correlation(reshaped_activations_A, reshaped_activations_B)

num_unq_pairs = len(list(set(corr_inds)))
print("% unique: ", num_unq_pairs / len(corr_inds))

sum(corr_vals) / len(corr_vals)

% unique:  0.24869791666666666


0.31679742952110246

In [42]:
sorted_feat_counts = Counter(corr_inds).most_common()
kept_modA_feats = [feat_ID for feat_ID, count in sorted_feat_counts if count == 1]
len(kept_modA_feats)

90

## save ds for features

In [47]:
def store_top_toks(top_acts_indices, top_acts_values, batch_tokens):
    feat_samps = []
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        new_str_token = tokenizer.decode(batch_tokens[batch_idx, seq_idx]).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
        feat_samps.append(new_str_token)
    return feat_samps

In [48]:
# store feature : lst of top strs
fList_model_B_top5 = []
samp_m = 5

for feature_idx in range(h_store_2.shape[-1]):
    if feature_idx % 500 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(h_store_2, feature_idx, samp_m, batch_tokens= inputs['input_ids'])
    fList_model_B_top5.append(store_top_toks(ds_top_acts_indices, ds_top_acts_values, inputs['input_ids']) )

Feature:  0
Feature:  500


In [58]:
# store feature : lst of top strs
fList_model_B_top30 = []
samp_m = 30

for feature_idx in range(h_store_2.shape[-1]):
    if feature_idx % 500 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(h_store_2, feature_idx, samp_m, batch_tokens= inputs['input_ids'])
    fList_model_B_top30.append(store_top_toks(ds_top_acts_indices, ds_top_acts_values, inputs['input_ids']) )

Feature:  0
Feature:  500


In [92]:
# store feature : lst of top strs
fList_model_B_top90 = []
samp_m = 90

for feature_idx in range(h_store_2.shape[-1]):
    if feature_idx % 500 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(h_store_2, feature_idx, samp_m, batch_tokens= inputs['input_ids'])
    fList_model_B_top90.append(store_top_toks(ds_top_acts_indices, ds_top_acts_values, inputs['input_ids']) )

Feature:  0
Feature:  500


## see the top dataset examps

In [126]:
fList_model_B_top30

[['.',
  'Exec',
  '.',
  '.',
  '.',
  '\\n',
  '.',
  '.',
  '.',
  '.',
  '.',
  '\\n',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.'],
 ['.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.'],
 ['.',
  '\\n',
  '.',
  '\\n',
  '.',
  '.',
  '\\n',
  '.',
  '.',
  '.',
  '.',
  '\\n',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.'],
 [' Mark',
  'Dave',
  ' Daniel',
  ' a',
  '.',
  ' Lou',
  ' Qu',
  ' Dave',
  'Sem',
  'Ad',
  ' Frank',
  ' Matt',
  '.',
  ' Frank',
  ' create',
  ' James',
  ' Frank',
  ' Jeremy',
  'i',
  ' support',
  ' him',
  ' comic',
  ' 1934',
  ' Cad',
  ' den',
  '.',
  ' of',
  ' Korea',
  ' Pro',
  ' Bud'],
 [' Group',
  ' groups',
  ' giant',
  ' 

## save spliced labels

In [ ]:
from google.colab import files

In [ ]:
with open('feat_snip_dict_B.pkl', 'wb') as f:
    pickle.dump(feat_snip_dict, f)
files.download('feat_snip_dict_B.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# with open('fList_model_A.pkl', 'wb') as f:
#     pickle.dump(fList_model_A, f)
# files.download('fList_model_A.pkl')

with open('fList_model_B.pkl', 'wb') as f:
    pickle.dump(fList_model_B, f)
files.download('fList_model_B.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## load

In [ ]:
import pickle
# with open('fList_model_A.pkl', 'rb') as f:
#     fList_model_A = pickle.load(f)
with open('fList_model_B.pkl', 'rb') as f:
    fList_model_B = pickle.load(f)

APPR1: match by most corr pair

In [ ]:
samp_m = 5

# combos of numerics- top 5

In [50]:
mixed_modA_feats, mixed_modB_feats = get_mixed_feats(fList_model_B_top5, corr_inds, keywords['numeric'])

Unique modA feats:  23
Unique modB feats:  31


In [51]:
get_new_mean_corr(mixed_modA_feats, mixed_modB_feats, corr_vals)

0.2760127238605333

In [52]:
X_subset = weight_matrix_np[mixed_modA_feats, :]
Y_subset = weight_matrix_2[mixed_modB_feats, :]

paired_svcca = svcca(X_subset, Y_subset, "nd")
paired_svcca

0.08190973439141587

In [53]:
num_feats = len(list(set(mixed_modA_feats)))
rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                num_feats, corr_inds, svcca, True)
print("Rand_corr SVCCA mean: ", sum(rand_corr_scores) / len(rand_corr_scores) )
p_value = np.mean(np.array(rand_corr_scores) >= paired_svcca)
print("P value: ", p_value)

Rand_corr SVCCA mean:  0.17837474651233648
P value:  0.77


In [54]:
rand_shuff_scores = shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                svcca, shapereq_bool=True)
print("Rand_shuff SVCCA mean: ", sum(rand_shuff_scores) / len(rand_shuff_scores) )
p_value = np.mean(np.array(rand_shuff_scores) >= paired_svcca)
print("P value: ", p_value)

Rand_shuff SVCCA mean:  0.14305520720464934
P value:  0.9


In [55]:
paired_rsa = representational_similarity_analysis(X_subset, Y_subset, "nd")
paired_rsa

0.004122875451527804

In [56]:
num_feats = len(list(set(mixed_modA_feats)))
rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                num_feats, corr_inds, representational_similarity_analysis, True)
print("Rand_corr RSA mean: ", sum(rand_corr_scores) / len(rand_corr_scores) )
p_value = np.mean(np.array(rand_corr_scores) >= paired_rsa)
print("P value: ", p_value)

Rand_corr RSA mean:  0.014954961060951222
P value:  0.48


In [57]:
rand_shuff_scores = shuffle_rand(100, X_subset, Y_subset, Y_subset.shape[0],
                                representational_similarity_analysis, shapereq_bool=True)
print("Rand_shuff RSA mean: ", sum(rand_shuff_scores) / len(rand_shuff_scores) )
p_value = np.mean(np.array(rand_shuff_scores) >= paired_rsa)
print("P value: ", p_value)

Rand_shuff RSA mean:  -0.001895704175173567
P value:  0.47


### 1-1 only

In [ ]:
filt_corr_ind_A, filt_corr_ind_B = filter_corr_pairs(mixed_modA_feats, mixed_modB_feats, kept_modA_feats)

% unique:  0.9658119658119658
num feats after filt:  234


In [ ]:
get_new_mean_corr(filt_corr_ind_A, filt_corr_ind_B, corr_vals)

0.42947734737422616

In [ ]:
X_subset = weight_matrix_np[filt_corr_ind_A]
Y_subset = weight_matrix_2[filt_corr_ind_B]

paired_svcca = svcca(X_subset, Y_subset, "nd")
paired_svcca

0.42471498563417676

In [ ]:
num_feats = len(list(set(filt_corr_ind_A)))
rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                num_feats, corr_inds, svcca, True)
print("Rand_corr SVCCA mean: ", sum(rand_corr_scores) / len(rand_corr_scores) )
p_value = np.mean(np.array(rand_corr_scores) >= paired_svcca)
print("P value: ", p_value)

Rand_corr SVCCA mean:  0.06475989702254895
P value:  0.0


In [ ]:
rand_shuff_scores = shuffle_rand(100, X_subset, Y_subset, Y_subset.shape[0],
                                svcca, shapereq_bool=True)
print("Rand_shuff SVCCA mean: ", sum(rand_shuff_scores) / len(rand_shuff_scores) )
p_value = np.mean(np.array(rand_shuff_scores) >= paired_svcca)
print("P value: ", p_value)

Rand_shuff SVCCA mean:  0.053623941776683066
P value:  0.0


In [ ]:
paired_rsa = representational_similarity_analysis(X_subset, Y_subset, "nd")
paired_rsa

0.05770592310046603

In [ ]:
num_feats = len(list(set(filt_corr_ind_A)))
rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                num_feats, corr_inds, representational_similarity_analysis, True)
print("Rand_corr RSA mean: ", sum(rand_corr_scores) / len(rand_corr_scores) )
p_value = np.mean(np.array(rand_corr_scores) >= paired_rsa)
print("P value: ", p_value)

Rand_corr RSA mean:  0.02953192870006762
P value:  0.02


In [ ]:
rand_shuff_scores = shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                representational_similarity_analysis, shapereq_bool=True)
print("Rand_shuff RSA mean: ", sum(rand_shuff_scores) / len(rand_shuff_scores) )
p_value = np.mean(np.array(rand_shuff_scores) >= paired_rsa)
print("P value: ", p_value)

Rand_shuff RSA mean:  0.001127662577389425
P value:  0.0


# combos of numerics- top 30

In [75]:
mixed_modA_feats, mixed_modB_feats = get_mixed_feats(fList_model_B_top30, corr_inds, keywords['numeric'])

Unique modA feats:  64
Unique modB feats:  135


In [76]:
get_new_mean_corr(mixed_modA_feats, mixed_modB_feats, corr_vals)

0.23202291945926845

In [77]:
X_subset = weight_matrix_np[mixed_modA_feats, :]
Y_subset = weight_matrix_2[mixed_modB_feats, :]

paired_svcca = svcca(X_subset, Y_subset, "nd")
paired_svcca

0.19198195041955743

In [78]:
num_feats = len(list(set(mixed_modA_feats)))
rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                num_feats, corr_inds, svcca, True)
print("Rand_corr SVCCA mean: ", sum(rand_corr_scores) / len(rand_corr_scores) )
p_value = np.mean(np.array(rand_corr_scores) >= paired_svcca)
print("P value: ", p_value)

Rand_corr SVCCA mean:  0.1166691710487302
P value:  0.19


In [91]:
rand_shuff_scores = shuffle_rand(50, X_subset, Y_subset, Y_subset.shape[0],
                                svcca, shapereq_bool=True)
print("Rand_shuff SVCCA mean: ", sum(rand_shuff_scores) / len(rand_shuff_scores) )
p_value = np.mean(np.array(rand_shuff_scores) >= paired_svcca)
print("P value: ", p_value)

Rand_shuff SVCCA mean:  0.08223665883106078
P value:  0.94


In [80]:
paired_rsa = representational_similarity_analysis(X_subset, Y_subset, "nd")
paired_rsa

-0.0017167316931660113

In [81]:
num_feats = len(list(set(mixed_modA_feats)))
rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                num_feats, corr_inds, representational_similarity_analysis, True)
print("Rand_corr RSA mean: ", sum(rand_corr_scores) / len(rand_corr_scores) )
p_value = np.mean(np.array(rand_corr_scores) >= paired_rsa)
print("P value: ", p_value)

Rand_corr RSA mean:  0.01657757746784611
P value:  0.72


In [82]:
rand_shuff_scores = shuffle_rand(100, X_subset, Y_subset, Y_subset.shape[0],
                                representational_similarity_analysis, shapereq_bool=True)
print("Rand_shuff RSA mean: ", sum(rand_shuff_scores) / len(rand_shuff_scores) )
p_value = np.mean(np.array(rand_shuff_scores) >= paired_rsa)
print("P value: ", p_value)

Rand_shuff RSA mean:  -0.001296923206644224
P value:  0.54


### 1-1 only

In [83]:
filt_corr_ind_A, filt_corr_ind_B = filter_corr_pairs(mixed_modA_feats, mixed_modB_feats, kept_modA_feats)

% unique:  0.810126582278481
num 1-1 feats after filt:  64


In [84]:
get_new_mean_corr(filt_corr_ind_A, filt_corr_ind_B, corr_vals)

0.23202291945926845

In [85]:
X_subset = weight_matrix_np[filt_corr_ind_A]
Y_subset = weight_matrix_2[filt_corr_ind_B]

paired_svcca = svcca(X_subset, Y_subset, "nd")
paired_svcca

0.007454594724854004

In [86]:
num_feats = len(list(set(filt_corr_ind_A)))
rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                num_feats, corr_inds, svcca, True)
print("Rand_corr SVCCA mean: ", sum(rand_corr_scores) / len(rand_corr_scores) )
p_value = np.mean(np.array(rand_corr_scores) >= paired_svcca)
print("P value: ", p_value)

Rand_corr SVCCA mean:  0.1168194673838135
P value:  0.98


In [87]:
rand_shuff_scores = shuffle_rand(100, X_subset, Y_subset, Y_subset.shape[0],
                                svcca, shapereq_bool=True)
print("Rand_shuff SVCCA mean: ", sum(rand_shuff_scores) / len(rand_shuff_scores) )
p_value = np.mean(np.array(rand_shuff_scores) >= paired_svcca)
print("P value: ", p_value)

Rand_shuff SVCCA mean:  0.08826971859357807
P value:  0.89


In [88]:
paired_rsa = representational_similarity_analysis(X_subset, Y_subset, "nd")
paired_rsa

0.020449192993492

In [89]:
num_feats = len(list(set(filt_corr_ind_A)))
rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                num_feats, corr_inds, representational_similarity_analysis, True)
print("Rand_corr RSA mean: ", sum(rand_corr_scores) / len(rand_corr_scores) )
p_value = np.mean(np.array(rand_corr_scores) >= paired_rsa)
print("P value: ", p_value)

Rand_corr RSA mean:  0.026685413808556574
P value:  0.56


In [90]:
rand_shuff_scores = shuffle_rand(10, X_subset, Y_subset, Y_subset.shape[0],
                                representational_similarity_analysis, shapereq_bool=True)
print("Rand_shuff RSA mean: ", sum(rand_shuff_scores) / len(rand_shuff_scores) )
p_value = np.mean(np.array(rand_shuff_scores) >= paired_rsa)
print("P value: ", p_value)

Rand_shuff RSA mean:  0.004190159182836681
P value:  0.2


# combos of numerics- top 90

In [93]:
mixed_modA_feats, mixed_modB_feats = get_mixed_feats(fList_model_B_top90, corr_inds, keywords['numeric'])

Unique modA feats:  117
Unique modB feats:  288


In [94]:
get_new_mean_corr(mixed_modA_feats, mixed_modB_feats, corr_vals)

0.22191314087209538

In [95]:
X_subset = weight_matrix_np[mixed_modA_feats, :]
Y_subset = weight_matrix_2[mixed_modB_feats, :]

paired_svcca = svcca(X_subset, Y_subset, "nd")
paired_svcca

0.22300861827935944

In [96]:
num_feats = len(list(set(mixed_modA_feats)))
rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                num_feats, corr_inds, svcca, True)
print("Rand_corr SVCCA mean: ", sum(rand_corr_scores) / len(rand_corr_scores) )
p_value = np.mean(np.array(rand_corr_scores) >= paired_svcca)
print("P value: ", p_value)

Rand_corr SVCCA mean:  0.11610207393004268
P value:  0.12


In [97]:
rand_shuff_scores = shuffle_rand(50, X_subset, Y_subset, Y_subset.shape[0],
                                svcca, shapereq_bool=True)
print("Rand_shuff SVCCA mean: ", sum(rand_shuff_scores) / len(rand_shuff_scores) )
p_value = np.mean(np.array(rand_shuff_scores) >= paired_svcca)
print("P value: ", p_value)

Rand_shuff SVCCA mean:  0.027127749088894974
P value:  0.0


In [98]:
paired_rsa = representational_similarity_analysis(X_subset, Y_subset, "nd")
paired_rsa

0.0012537334420515856

In [99]:
num_feats = len(list(set(mixed_modA_feats)))
rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                num_feats, corr_inds, representational_similarity_analysis, True)
print("Rand_corr RSA mean: ", sum(rand_corr_scores) / len(rand_corr_scores) )
p_value = np.mean(np.array(rand_corr_scores) >= paired_rsa)
print("P value: ", p_value)

Rand_corr RSA mean:  0.019574649999213517
P value:  0.85


In [100]:
rand_shuff_scores = shuffle_rand(100, X_subset, Y_subset, Y_subset.shape[0],
                                representational_similarity_analysis, shapereq_bool=True)
print("Rand_shuff RSA mean: ", sum(rand_shuff_scores) / len(rand_shuff_scores) )
p_value = np.mean(np.array(rand_shuff_scores) >= paired_rsa)
print("P value: ", p_value)

Rand_shuff RSA mean:  -0.00026078779538652385
P value:  0.22


### 1-1 only

In [101]:
filt_corr_ind_A, filt_corr_ind_B = filter_corr_pairs(mixed_modA_feats, mixed_modB_feats, kept_modA_feats)

% unique:  0.7090909090909091
num 1-1 feats after filt:  117


In [102]:
get_new_mean_corr(filt_corr_ind_A, filt_corr_ind_B, corr_vals)

0.22191314087209538

In [103]:
X_subset = weight_matrix_np[filt_corr_ind_A]
Y_subset = weight_matrix_2[filt_corr_ind_B]

paired_svcca = svcca(X_subset, Y_subset, "nd")
paired_svcca

0.06160376295450926

In [104]:
num_feats = len(list(set(filt_corr_ind_A)))
rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                num_feats, corr_inds, svcca, True)
print("Rand_corr SVCCA mean: ", sum(rand_corr_scores) / len(rand_corr_scores) )
p_value = np.mean(np.array(rand_corr_scores) >= paired_svcca)
print("P value: ", p_value)

Rand_corr SVCCA mean:  0.11566442969219691
P value:  0.76


In [105]:
rand_shuff_scores = shuffle_rand(100, X_subset, Y_subset, Y_subset.shape[0],
                                svcca, shapereq_bool=True)
print("Rand_shuff SVCCA mean: ", sum(rand_shuff_scores) / len(rand_shuff_scores) )
p_value = np.mean(np.array(rand_shuff_scores) >= paired_svcca)
print("P value: ", p_value)

Rand_shuff SVCCA mean:  0.05911455916918389
P value:  0.33


In [106]:
paired_rsa = representational_similarity_analysis(X_subset, Y_subset, "nd")
paired_rsa

0.05406381440284956

In [111]:
num_feats = len(list(set(filt_corr_ind_A)))
rand_corr_scores = score_rand_corr(1000, weight_matrix_np, weight_matrix_2,
                num_feats, corr_inds, representational_similarity_analysis, True)
print("Rand_corr RSA mean: ", sum(rand_corr_scores) / len(rand_corr_scores) )
p_value = np.mean(np.array(rand_corr_scores) >= paired_rsa)
print("P value: ", p_value)

Rand_corr RSA mean:  0.01907907486523717
P value:  0.042


In [113]:
rand_shuff_scores = shuffle_rand(1000, X_subset, Y_subset, Y_subset.shape[0],
                                representational_similarity_analysis, shapereq_bool=True)
print("Rand_shuff RSA mean: ", sum(rand_shuff_scores) / len(rand_shuff_scores) )
p_value = np.mean(np.array(rand_shuff_scores) >= paired_rsa)
print("P value: ", p_value)

Rand_shuff RSA mean:  1.986811569328785e-05
P value:  0.0


# loop for one layer pair: top 90

In [114]:
metric_dict = {'SVCCA': svcca, 'RSA': representational_similarity_analysis}
# map_type_list = ['manyTo1', '1To1']
map_type_list = ['1To1']

all_scores_dict = {}
for concept_name in keywords.keys():
    print(concept_name)
    scores_dict = {}

    mixed_modA_feats, mixed_modB_feats = get_mixed_feats(fList_model_B_top90, corr_inds, keywords[concept_name])
    filt_corr_ind_A, filt_corr_ind_B = filter_corr_pairs(mixed_modA_feats, mixed_modB_feats, kept_modA_feats)

    for map_type in map_type_list:
        print(map_type)
        if map_type == '1To1':
            X_subset = weight_matrix_np[filt_corr_ind_A]
            Y_subset = weight_matrix_2[filt_corr_ind_B]
            num_feats = len(list(set(filt_corr_ind_A)))
        else:
            X_subset = weight_matrix_np[mixed_modA_feats]
            Y_subset = weight_matrix_2[mixed_modB_feats]
            num_feats = len(list(set(mixed_modA_feats)))

        scores_dict[f'mean_corr_{map_type}'] = get_new_mean_corr(mixed_modA_feats, mixed_modB_feats, corr_vals)

        for metric_name, metric_fn in metric_dict.items():
            paired_score = metric_fn(X_subset, Y_subset, "nd")
            scores_dict[f'paired_{metric_name}_{map_type}'] = paired_score

            rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                            num_feats, corr_inds, metric_fn, True)
            scores_dict[f'rand_corr_mean_{metric_name}_{map_type}'] = sum(rand_corr_scores) / len(rand_corr_scores)
            scores_dict[f'rand_corr_pval_{metric_name}_{map_type}'] =  np.mean(np.array(rand_corr_scores) >= paired_score)

            rand_shuff_scores = shuffle_rand(100, X_subset, Y_subset, Y_subset.shape[0],
                                            metric_fn, shapereq_bool=True)
            scores_dict[f'rand_shuff_mean_{metric_name}_{map_type}'] = sum(rand_shuff_scores) / len(rand_shuff_scores)
            scores_dict[f'rand_shuff_pval_{metric_name}_{map_type}'] =  np.mean(np.array(rand_shuff_scores) >= paired_score)

    all_scores_dict[concept_name] = scores_dict
    print('\n')

numeric
Unique modA feats:  117
Unique modB feats:  288
% unique:  0.7090909090909091
num 1-1 feats after filt:  117
1To1


people
Unique modA feats:  76
Unique modB feats:  142
% unique:  0.9047619047619048
num 1-1 feats after filt:  76
1To1


nature
Unique modA feats:  77
Unique modB feats:  135
% unique:  0.9506172839506173
num 1-1 feats after filt:  77
1To1


science
Unique modA feats:  132
Unique modB feats:  344
% unique:  0.8461538461538461
num 1-1 feats after filt:  132
1To1


animals
Unique modA feats:  26
Unique modB feats:  46
% unique:  1.0
num 1-1 feats after filt:  26
1To1




## analyze data

In [115]:
import pandas as pd
from IPython.display import display

In [116]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.250023
nature,0.220464
numeric,0.221913
people,0.218943
science,0.220239


In [117]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.06,0.11,0.73,0.06,0.36
people,0.07,0.12,0.71,0.10,0.65
nature,0.01,0.11,0.95,0.09,0.96
science,0.11,0.11,0.44,0.06,0.14
animals,0.18,0.16,0.40,0.16,0.38


In [118]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.05,0.02,0.04,-0.00,0.00
people,0.02,0.02,0.42,-0.00,0.04
nature,-0.00,0.02,0.79,-0.00,0.43
science,0.02,0.02,0.41,-0.00,0.00
animals,-0.00,0.02,0.58,0.01,0.58


# loop for one layer pair: top 30

In [127]:
metric_dict = {'SVCCA': svcca, 'RSA': representational_similarity_analysis}
# map_type_list = ['manyTo1', '1To1']
map_type_list = ['1To1']

all_scores_dict = {}
for concept_name in keywords.keys():
    print(concept_name)
    scores_dict = {}

    mixed_modA_feats, mixed_modB_feats = get_mixed_feats(fList_model_B_top30, corr_inds, keywords[concept_name])
    filt_corr_ind_A, filt_corr_ind_B = filter_corr_pairs(mixed_modA_feats, mixed_modB_feats, kept_modA_feats)

    for map_type in map_type_list:
        print(map_type)
        if map_type == '1To1':
            X_subset = weight_matrix_np[filt_corr_ind_A]
            Y_subset = weight_matrix_2[filt_corr_ind_B]
            num_feats = len(list(set(filt_corr_ind_A)))
        else:
            X_subset = weight_matrix_np[mixed_modA_feats]
            Y_subset = weight_matrix_2[mixed_modB_feats]
            num_feats = len(list(set(mixed_modA_feats)))

        scores_dict[f'mean_corr_{map_type}'] = get_new_mean_corr(mixed_modA_feats, mixed_modB_feats, corr_vals)

        for metric_name, metric_fn in metric_dict.items():
            paired_score = metric_fn(X_subset, Y_subset, "nd")
            scores_dict[f'paired_{metric_name}_{map_type}'] = paired_score

            rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                            num_feats, corr_inds, metric_fn, True)
            scores_dict[f'rand_corr_mean_{metric_name}_{map_type}'] = sum(rand_corr_scores) / len(rand_corr_scores)
            scores_dict[f'rand_corr_pval_{metric_name}_{map_type}'] =  np.mean(np.array(rand_corr_scores) >= paired_score)

            rand_shuff_scores = shuffle_rand(100, X_subset, Y_subset, Y_subset.shape[0],
                                            metric_fn, shapereq_bool=True)
            scores_dict[f'rand_shuff_mean_{metric_name}_{map_type}'] = sum(rand_shuff_scores) / len(rand_shuff_scores)
            scores_dict[f'rand_shuff_pval_{metric_name}_{map_type}'] =  np.mean(np.array(rand_shuff_scores) >= paired_score)

    all_scores_dict[concept_name] = scores_dict
    print('\n')

numeric
Unique modA feats:  64
Unique modB feats:  135
% unique:  0.810126582278481
num 1-1 feats after filt:  64
1To1


people
Unique modA feats:  40
Unique modB feats:  57
% unique:  0.9090909090909091
num 1-1 feats after filt:  40
1To1


nature
Unique modA feats:  43
Unique modB feats:  61
% unique:  1.0
num 1-1 feats after filt:  43
1To1


science
Unique modA feats:  83
Unique modB feats:  204
% unique:  0.9120879120879121
num 1-1 feats after filt:  83
1To1


animals
Unique modA feats:  6
Unique modB feats:  11
% unique:  1.0
num 1-1 feats after filt:  6
1To1




## analyze data

In [128]:
import pandas as pd
from IPython.display import display

In [129]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.275903
nature,0.241440
numeric,0.232023
people,0.253129
science,0.233407


In [130]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.01,0.12,0.97,0.08,0.94
people,0.08,0.15,0.67,0.12,0.52
nature,0.00,0.15,1.00,0.13,0.99
science,0.09,0.13,0.66,0.08,0.31
animals,0.32,0.35,0.45,0.32,0.52


In [131]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.01,0.37,0.00,0.16
people,0.05,0.02,0.17,-0.00,0.01
nature,0.02,0.02,0.42,-0.00,0.25
science,-0.00,0.02,0.81,-0.00,0.61
animals,-0.21,-0.03,0.74,0.02,0.74


# loop for one layer pair: top 5

In [136]:
metric_dict = {'SVCCA': svcca, 'RSA': representational_similarity_analysis}
# map_type_list = ['manyTo1', '1To1']
map_type_list = ['1To1']

all_scores_dict = {}
for concept_name in keywords.keys():
    print(concept_name)
    scores_dict = {}

    mixed_modA_feats, mixed_modB_feats = get_mixed_feats(fList_model_B_top5, corr_inds, keywords[concept_name])
    filt_corr_ind_A, filt_corr_ind_B = filter_corr_pairs(mixed_modA_feats, mixed_modB_feats, kept_modA_feats)

    if len(list(set(filt_corr_ind_A))) <= 2:
        continue

    for map_type in map_type_list:
        print(map_type)
        if map_type == '1To1':
            X_subset = weight_matrix_np[filt_corr_ind_A]
            Y_subset = weight_matrix_2[filt_corr_ind_B]
            num_feats = len(list(set(filt_corr_ind_A)))
        else:
            X_subset = weight_matrix_np[mixed_modA_feats]
            Y_subset = weight_matrix_2[mixed_modB_feats]
            num_feats = len(list(set(mixed_modA_feats)))

        scores_dict[f'mean_corr_{map_type}'] = get_new_mean_corr(mixed_modA_feats, mixed_modB_feats, corr_vals)

        for metric_name, metric_fn in metric_dict.items():
            paired_score = metric_fn(X_subset, Y_subset, "nd")
            scores_dict[f'paired_{metric_name}_{map_type}'] = paired_score

            rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                            num_feats, corr_inds, metric_fn, True)
            scores_dict[f'rand_corr_mean_{metric_name}_{map_type}'] = sum(rand_corr_scores) / len(rand_corr_scores)
            scores_dict[f'rand_corr_pval_{metric_name}_{map_type}'] =  np.mean(np.array(rand_corr_scores) >= paired_score)

            rand_shuff_scores = shuffle_rand(100, X_subset, Y_subset, Y_subset.shape[0],
                                            metric_fn, shapereq_bool=True)
            scores_dict[f'rand_shuff_mean_{metric_name}_{map_type}'] = sum(rand_shuff_scores) / len(rand_shuff_scores)
            scores_dict[f'rand_shuff_pval_{metric_name}_{map_type}'] =  np.mean(np.array(rand_shuff_scores) >= paired_score)

    all_scores_dict[concept_name] = scores_dict
    print('\n')

numeric
Unique modA feats:  23
Unique modB feats:  31
% unique:  0.9583333333333334
num 1-1 feats after filt:  23
1To1


people
Unique modA feats:  14
Unique modB feats:  17
% unique:  1.0
num 1-1 feats after filt:  14
1To1


nature
Unique modA feats:  11
Unique modB feats:  11
% unique:  1.0
num 1-1 feats after filt:  11
1To1


science
Unique modA feats:  35
Unique modB feats:  60
% unique:  0.9722222222222222
num 1-1 feats after filt:  35
1To1


animals
Unique modA feats:  1
Unique modB feats:  1
% unique:  1.0
num 1-1 feats after filt:  1


## analyze data

In [138]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
nature,0.293456
numeric,0.276013
people,0.288157
science,0.251122


In [139]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.01,0.19,0.91,0.14,0.97
people,0.42,0.22,0.10,0.23,0.13
nature,0.20,0.28,0.61,0.29,0.58
science,0.02,0.13,0.94,0.16,0.92


In [140]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.01,0.01,0.49,-0.00,0.44
people,0.02,0.03,0.55,-0.00,0.40
nature,0.03,0.03,0.46,-0.01,0.39
science,-0.00,0.01,0.60,-0.00,0.54


# loop for any layer pair v L5

In [144]:
# def run_expm(layer_id, outputs, outputs_2):
def run_semantic_subspace_expms(fList_model_B, outputs):
    metric_dict = {'SVCCA': svcca, 'RSA': representational_similarity_analysis}
    # map_type_list = ['manyTo1', '1To1']
    map_type_list = ['1To1']
    layer_to_dictscores = {}

    name = "EleutherAI/sae-pythia-70m-32k"
    for layerID_2 in range(0, 6):
        print("Layer: " + str(layerID_2))
        dictscores = {}

        weight_matrix_np = model.gpt_neox.layers[layerID_2].mlp.dense_4h_to_h.weight
        weight_matrix_np = weight_matrix_np.cpu().detach().numpy()

        hidden_state = outputs.hidden_states[layerID_2].to("cuda")

        first_dim_reshaped = hidden_state.shape[0] * hidden_state.shape[1]
        reshaped_activations_A = hidden_state.reshape(first_dim_reshaped, hidden_state.shape[-1]).cpu()

        del hidden_state
        torch.cuda.empty_cache()

        corr_inds, corr_vals = batched_correlation(reshaped_activations_A, reshaped_activations_B)
        sorted_feat_counts = Counter(corr_inds).most_common()
        kept_modA_feats = [feat_ID for feat_ID, count in sorted_feat_counts if count == 1]

        all_scores_dict = {}
        for concept_name in keywords.keys():
            print(concept_name)
            scores_dict = {}

            mixed_modA_feats, mixed_modB_feats = get_mixed_feats(fList_model_B, corr_inds, keywords[concept_name])
            filt_corr_ind_A, filt_corr_ind_B = filter_corr_pairs(mixed_modA_feats, mixed_modB_feats, kept_modA_feats)

            for map_type in map_type_list:
                print(map_type)
                if map_type == '1To1':
                    X_subset = weight_matrix_np[filt_corr_ind_A]
                    Y_subset = weight_matrix_2[filt_corr_ind_B]
                    num_feats = len(list(set(filt_corr_ind_A)))
                else:
                    X_subset = weight_matrix_np[mixed_modA_feats]
                    Y_subset = weight_matrix_2[mixed_modB_feats]
                    num_feats = len(list(set(mixed_modA_feats)))

                scores_dict[f'mean_corr_{map_type}'] = get_new_mean_corr(mixed_modA_feats, mixed_modB_feats, corr_vals)

                for metric_name, metric_fn in metric_dict.items():
                    paired_score = metric_fn(X_subset, Y_subset, "nd")
                    scores_dict[f'paired_{metric_name}_{map_type}'] = paired_score

                    rand_corr_scores = score_rand_corr(100, weight_matrix_np, weight_matrix_2,
                                    num_feats, corr_inds, metric_fn, True)
                    scores_dict[f'rand_corr_mean_{metric_name}_{map_type}'] = sum(rand_corr_scores) / len(rand_corr_scores)
                    scores_dict[f'rand_corr_pval_{metric_name}_{map_type}'] =  np.mean(np.array(rand_corr_scores) >= paired_score)

                    rand_shuff_scores = shuffle_rand(100, X_subset, Y_subset, Y_subset.shape[0],
                                                    metric_fn, shapereq_bool=True)
                    scores_dict[f'rand_shuff_mean_{metric_name}_{map_type}'] = sum(rand_shuff_scores) / len(rand_shuff_scores)
                    scores_dict[f'rand_shuff_pval_{metric_name}_{map_type}'] =  np.mean(np.array(rand_shuff_scores) >= paired_score)

            all_scores_dict[concept_name] = scores_dict
            print('\n')

        # for key, value in all_scores_dict.items():
        #     print(key + ": " + str(value))
        # print("\n")

        layer_to_dictscores[layerID_2] = all_scores_dict
    return layer_to_dictscores

In [145]:
layer_to_dictscores = run_semantic_subspace_expms(fList_model_B_top30, outputs)

Layer: 0
numeric
Unique modA feats:  51
Unique modB feats:  135
% unique:  0.8360655737704918
num 1-1 feats after filt:  51
1To1


people
Unique modA feats:  34
Unique modB feats:  57
% unique:  1.0
num 1-1 feats after filt:  34
1To1


nature
Unique modA feats:  28
Unique modB feats:  61
% unique:  1.0
num 1-1 feats after filt:  28
1To1


science
Unique modA feats:  59
Unique modB feats:  204
% unique:  0.9365079365079365
num 1-1 feats after filt:  59
1To1


animals
Unique modA feats:  10
Unique modB feats:  11
% unique:  1.0
num 1-1 feats after filt:  10
1To1


Layer: 1
numeric
Unique modA feats:  50
Unique modB feats:  135
% unique:  0.8771929824561403
num 1-1 feats after filt:  50
1To1


people
Unique modA feats:  34
Unique modB feats:  57
% unique:  0.8947368421052632
num 1-1 feats after filt:  34
1To1


nature
Unique modA feats:  29
Unique modB feats:  61
% unique:  0.9666666666666667
num 1-1 feats after filt:  29
1To1


science
Unique modA feats:  58
Unique modB feats:  204
% uni

In [146]:
from google.colab import files

In [147]:
layer_id = 3
modeltype = 'pythia70m'
with open(f'layer_to_dictscores_L{layer_id}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(layer_to_dictscores, f)
files.download(f'layer_to_dictscores_L{layer_id}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## analyze data: L0

In [148]:
all_scores_dict = layer_to_dictscores[0]

In [149]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.150199
nature,0.161771
numeric,0.146051
people,0.152662
science,0.139209


In [150]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.96,0.13,0.00,0.10,0.00
people,0.15,0.17,0.55,0.13,0.38
nature,0.00,0.17,1.00,0.14,0.99
science,0.08,0.13,0.70,0.10,0.54
animals,0.29,0.28,0.39,0.30,0.50


In [151]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.07,-0.00,0.00,0.00,0.00
people,-0.06,-0.01,0.89,0.00,0.96
nature,0.01,-0.00,0.35,-0.00,0.42
science,-0.02,0.00,0.83,0.00,0.78
animals,-0.08,-0.01,0.69,-0.02,0.70


## analyze data: L1

In [152]:
all_scores_dict = layer_to_dictscores[1]

In [153]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.211720
nature,0.205841
numeric,0.196687
people,0.210433
science,0.193597


In [154]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.08,0.18,0.69,0.10,0.48
people,0.08,0.21,0.86,0.13,0.61
nature,0.03,0.19,0.95,0.14,0.82
science,0.14,0.18,0.59,0.09,0.24
animals,0.18,0.36,0.72,0.36,0.67


In [155]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.08,0.03,0.12,-0.0,0.00
people,0.02,0.04,0.61,-0.0,0.32
nature,0.06,0.03,0.29,0.0,0.15
science,0.04,0.04,0.52,-0.0,0.07
animals,0.10,0.06,0.46,-0.0,0.33


## analyze data: L2

In [156]:
all_scores_dict = layer_to_dictscores[2]

In [157]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.267506
nature,0.239772
numeric,0.227143
people,0.242267
science,0.234344


In [158]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.09,0.12,0.58,0.10,0.42
people,0.07,0.12,0.64,0.11,0.60
nature,0.06,0.18,0.87,0.13,0.78
science,0.02,0.12,0.87,0.09,0.81
animals,0.00,0.32,1.00,0.30,1.00


In [159]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.04,0.02,0.23,-0.0,0.02
people,0.06,0.01,0.09,0.0,0.04
nature,0.01,0.02,0.56,0.0,0.42
science,0.01,0.01,0.46,-0.0,0.24
animals,0.15,0.03,0.22,0.0,0.18


## analyze data: L3

In [160]:
all_scores_dict = layer_to_dictscores[3]

In [161]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.275903
nature,0.241440
numeric,0.232023
people,0.253129
science,0.233407


In [162]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.01,0.14,1.00,0.08,0.89
people,0.08,0.15,0.70,0.11,0.49
nature,0.00,0.12,0.99,0.12,1.00
science,0.09,0.10,0.45,0.07,0.23
animals,0.32,0.41,0.61,0.29,0.43


In [163]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.02,0.51,0.00,0.14
people,0.05,0.02,0.14,-0.00,0.05
nature,0.02,0.02,0.48,-0.00,0.27
science,-0.00,0.02,0.87,-0.00,0.45
animals,-0.21,0.03,0.78,-0.05,0.71


## analyze data: L4

In [164]:
all_scores_dict = layer_to_dictscores[4]

In [165]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.274582
nature,0.249255
numeric,0.235313
people,0.256783
science,0.227193


In [166]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.02,0.19,0.96,0.07,0.62
people,0.45,0.21,0.02,0.13,0.00
nature,0.11,0.21,0.81,0.12,0.54
science,0.18,0.18,0.50,0.09,0.14
animals,0.34,0.36,0.48,0.36,0.46


In [167]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.00,0.05,0.92,0.00,0.47
people,0.01,0.04,0.70,0.00,0.37
nature,0.06,0.04,0.36,0.00,0.04
science,0.03,0.04,0.67,0.00,0.02
animals,0.30,0.03,0.15,-0.04,0.12


## analyze data: L5

In [168]:
all_scores_dict = layer_to_dictscores[4]

In [169]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.274582
nature,0.249255
numeric,0.235313
people,0.256783
science,0.227193


In [170]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.02,0.19,0.96,0.07,0.62
people,0.45,0.21,0.02,0.13,0.00
nature,0.11,0.21,0.81,0.12,0.54
science,0.18,0.18,0.50,0.09,0.14
animals,0.34,0.36,0.48,0.36,0.46


In [171]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.00,0.05,0.92,0.00,0.47
people,0.01,0.04,0.70,0.00,0.37
nature,0.06,0.04,0.36,0.00,0.04
science,0.03,0.04,0.67,0.00,0.02
animals,0.30,0.03,0.15,-0.04,0.12


# loop for any layer pair v L10

In [175]:
mod_B_layer = 10

weight_matrix_2 = model_2.gpt_neox.layers[mod_B_layer].mlp.dense_4h_to_h.weight
weight_matrix_2 = weight_matrix_2.cpu().detach().numpy()

with torch.inference_mode():
    hidden_state_2 = outputs_2.hidden_states[mod_B_layer].to("cuda")

reshaped_activations_B = hidden_state_2.reshape(first_dim_reshaped, hidden_state_2.shape[-1]).cpu()
del hidden_state_2
torch.cuda.empty_cache()

In [176]:
h_store_2 = outputs_2.hidden_states[mod_B_layer].to("cuda")
fList_model_B_top30 = []
samp_m = 30

for feature_idx in range(h_store_2.shape[-1]):
    if feature_idx % 500 == 0:
        print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(h_store_2, feature_idx, samp_m, batch_tokens= inputs['input_ids'])
    fList_model_B_top30.append(store_top_toks(ds_top_acts_indices, ds_top_acts_values, inputs['input_ids']) )

Feature:  0
Feature:  500


In [177]:
layer_to_dictscores = run_semantic_subspace_expms(fList_model_B_top30, outputs)

Layer: 0
numeric
Unique modA feats:  47
Unique modB feats:  165
% unique:  0.9215686274509803
num 1-1 feats after filt:  47
1To1


people
Unique modA feats:  20
Unique modB feats:  30
% unique:  1.0
num 1-1 feats after filt:  20
1To1


nature
Unique modA feats:  23
Unique modB feats:  34
% unique:  1.0
num 1-1 feats after filt:  23
1To1


science
Unique modA feats:  36
Unique modB feats:  135
% unique:  0.972972972972973
num 1-1 feats after filt:  36
1To1


animals
Unique modA feats:  2
Unique modB feats:  3
% unique:  1.0
num 1-1 feats after filt:  2
1To1


<ipython-input-12-e01a87a701bd>:419: RuntimeWarning: invalid value encountered in divide
  explained_variance = (singular_values**2) / total_variance
<ipython-input-12-e01a87a701bd>:292: RuntimeWarning: invalid value encountered in divide
  sigmaxx /= xmax
<ipython-input-12-e01a87a701bd>:294: RuntimeWarning: invalid value encountered in divide
  sigmaxy /= np.sqrt(xmax * ymax)
<ipython-input-12-e01a87a701bd>:295: RuntimeWarning: invalid value encountered in divide
  sigmayx /= np.sqrt(xmax * ymax)




Layer: 1
numeric
Unique modA feats:  55
Unique modB feats:  165
% unique:  0.9482758620689655
num 1-1 feats after filt:  55
1To1


people
Unique modA feats:  17
Unique modB feats:  30
% unique:  1.0
num 1-1 feats after filt:  17
1To1


nature
Unique modA feats:  20
Unique modB feats:  34
% unique:  0.9523809523809523
num 1-1 feats after filt:  20
1To1


science
Unique modA feats:  37
Unique modB feats:  135
% unique:  0.9487179487179487
num 1-1 feats after filt:  37
1To1


animals
Unique modA feats:  2
Unique modB feats:  3
% unique:  1.0
num 1-1 feats after filt:  2
1To1


<ipython-input-12-e01a87a701bd>:419: RuntimeWarning: invalid value encountered in divide
  explained_variance = (singular_values**2) / total_variance
<ipython-input-12-e01a87a701bd>:292: RuntimeWarning: invalid value encountered in divide
  sigmaxx /= xmax
<ipython-input-12-e01a87a701bd>:294: RuntimeWarning: invalid value encountered in divide
  sigmaxy /= np.sqrt(xmax * ymax)
<ipython-input-12-e01a87a701bd>:295: RuntimeWarning: invalid value encountered in divide
  sigmayx /= np.sqrt(xmax * ymax)




Layer: 2
numeric
Unique modA feats:  74
Unique modB feats:  165
% unique:  0.8705882352941177
num 1-1 feats after filt:  74
1To1


people
Unique modA feats:  21
Unique modB feats:  30
% unique:  1.0
num 1-1 feats after filt:  21
1To1


nature
Unique modA feats:  24
Unique modB feats:  34
% unique:  0.96
num 1-1 feats after filt:  24
1To1


science
Unique modA feats:  50
Unique modB feats:  135
% unique:  0.9259259259259259
num 1-1 feats after filt:  50
1To1


animals
Unique modA feats:  2
Unique modB feats:  3
% unique:  1.0
num 1-1 feats after filt:  2
1To1


<ipython-input-12-e01a87a701bd>:419: RuntimeWarning: invalid value encountered in divide
  explained_variance = (singular_values**2) / total_variance
<ipython-input-12-e01a87a701bd>:292: RuntimeWarning: invalid value encountered in divide
  sigmaxx /= xmax
<ipython-input-12-e01a87a701bd>:294: RuntimeWarning: invalid value encountered in divide
  sigmaxy /= np.sqrt(xmax * ymax)
<ipython-input-12-e01a87a701bd>:295: RuntimeWarning: invalid value encountered in divide
  sigmayx /= np.sqrt(xmax * ymax)




Layer: 3
numeric
Unique modA feats:  86
Unique modB feats:  165
% unique:  0.8865979381443299
num 1-1 feats after filt:  86
1To1


people
Unique modA feats:  22
Unique modB feats:  30
% unique:  1.0
num 1-1 feats after filt:  22
1To1


nature
Unique modA feats:  27
Unique modB feats:  34
% unique:  1.0
num 1-1 feats after filt:  27
1To1


science
Unique modA feats:  65
Unique modB feats:  135
% unique:  0.9558823529411765
num 1-1 feats after filt:  65
1To1


animals
Unique modA feats:  3
Unique modB feats:  3
% unique:  1.0
num 1-1 feats after filt:  3
1To1


Layer: 4
numeric
Unique modA feats:  98
Unique modB feats:  165
% unique:  0.8828828828828829
num 1-1 feats after filt:  98
1To1


people
Unique modA feats:  25
Unique modB feats:  30
% unique:  1.0
num 1-1 feats after filt:  25
1To1


nature
Unique modA feats:  25
Unique modB feats:  34
% unique:  0.9615384615384616
num 1-1 feats after filt:  25
1To1


science
Unique modA feats:  70
Unique modB feats:  135
% unique:  0.98591549

In [178]:
from google.colab import files

In [179]:
modeltype = 'pythia160m'
with open(f'layer_to_dictscores_L{mod_B_layer}_{modeltype}.pkl', 'wb') as f:
    pickle.dump(layer_to_dictscores, f)
files.download(f'layer_to_dictscores_L{mod_B_layer}_{modeltype}.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## analyze data: L0

In [180]:
all_scores_dict = layer_to_dictscores[0]

In [181]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.362698
nature,0.173711
numeric,0.139988
people,0.169852
science,0.170020


In [182]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.66,0.11,0.00,0.10,0.00
people,0.03,0.18,0.91,0.15,0.84
nature,0.27,0.17,0.16,0.17,0.17
science,0.97,0.14,0.00,0.08,0.00
animals,1.00,1.00,0.83,1.00,1.00


In [183]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.00,0.19,-0.0,0.17
people,-0.02,-0.01,0.56,-0.0,0.56
nature,-0.06,-0.00,0.83,0.0,0.80
science,0.05,0.00,0.16,0.0,0.11
animals,NaN,NaN,0.00,NaN,0.00


## analyze data: L1

In [184]:
all_scores_dict = layer_to_dictscores[1]

In [185]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.421982
nature,0.230338
numeric,0.163088
people,0.212055
science,0.193807


In [186]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.03,0.13,0.85,0.09,0.74
people,0.22,0.26,0.40,0.19,0.36
nature,0.06,0.24,0.81,0.16,0.71
science,0.02,0.17,0.90,0.10,0.74
animals,1.00,1.00,0.91,1.00,1.00


In [187]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.06,-0.00,0.00,0.00,0.00
people,-0.00,-0.02,0.42,-0.00,0.44
nature,0.04,-0.00,0.29,-0.01,0.18
science,-0.03,0.01,0.78,-0.00,0.77
animals,NaN,NaN,0.00,NaN,0.00


## analyze data: L2

In [188]:
all_scores_dict = layer_to_dictscores[2]

In [189]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.427948
nature,0.240717
numeric,0.187709
people,0.237699
science,0.217894


In [190]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.09,0.08,0.36,0.05,0.07
people,0.05,0.17,0.83,0.11,0.63
nature,0.00,0.13,0.97,0.11,1.00
science,0.00,0.11,0.98,0.07,0.94
animals,1.00,1.00,0.85,1.00,1.00


In [191]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.01,0.00,0.26,-0.00,0.19
people,0.07,-0.00,0.09,0.01,0.21
nature,-0.02,0.01,0.69,-0.00,0.56
science,0.03,0.00,0.16,-0.00,0.14
animals,NaN,NaN,0.00,NaN,0.00


## analyze data: L3

In [192]:
all_scores_dict = layer_to_dictscores[3]

In [193]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.355847
nature,0.243481
numeric,0.197058
people,0.210466
science,0.218857


In [194]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.01,0.09,0.91,0.05,0.85
people,0.04,0.18,0.84,0.14,0.74
nature,0.10,0.13,0.37,0.15,0.55
science,0.04,0.10,0.55,0.07,0.44
animals,1.00,0.65,0.05,0.64,0.12


In [195]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.01,0.00,0.38,-0.00,0.17
people,0.04,0.01,0.28,0.00,0.30
nature,-0.05,-0.01,0.77,0.00,0.80
science,-0.01,0.00,0.76,-0.00,0.70
animals,-0.50,0.01,0.90,-0.11,0.78


## analyze data: L4

In [196]:
all_scores_dict = layer_to_dictscores[4]

In [197]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.351483
nature,0.249549
numeric,0.213382
people,0.239675
science,0.243206


In [198]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.03,0.08,0.67,0.05,0.48
people,0.04,0.12,0.80,0.07,0.71
nature,0.06,0.16,0.69,0.08,0.50
science,0.04,0.09,0.52,0.03,0.25
animals,0.99,0.70,0.16,0.65,0.15


In [199]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.01,0.19,-0.00,0.00
people,-0.02,0.01,0.66,0.01,0.67
nature,0.07,0.00,0.12,0.01,0.14
science,0.00,0.01,0.69,0.00,0.54
animals,0.50,0.10,0.55,0.08,0.51


## analyze data: L5

In [200]:
all_scores_dict = layer_to_dictscores[4]

In [201]:
filtered_words = ['mean_corr']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index')
display(df)

,mean_corr_1To1
animals,0.351483
nature,0.249549
numeric,0.213382
people,0.239675
science,0.243206


In [202]:
filtered_words = ['1To1', 'SVCCA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
# Convert the filtered data into a DataFrame, with each key as a row
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_SVCCA_1To1,rand_corr_mean_SVCCA_1To1,rand_corr_pval_SVCCA_1To1,rand_shuff_mean_SVCCA_1To1,rand_shuff_pval_SVCCA_1To1
numeric,0.03,0.08,0.67,0.05,0.48
people,0.04,0.12,0.80,0.07,0.71
nature,0.06,0.16,0.69,0.08,0.50
science,0.04,0.09,0.52,0.03,0.25
animals,0.99,0.70,0.16,0.65,0.15


In [203]:
filtered_words = ['1To1', 'RSA']
filtered_data = {
    key: {k: v for k, v in sub_dict.items() if all(word in k for word in filtered_words)}
    for key, sub_dict in all_scores_dict.items()
}
df = pd.DataFrame.from_dict(filtered_data, orient='index').round(2)
display(df)

,paired_RSA_1To1,rand_corr_mean_RSA_1To1,rand_corr_pval_RSA_1To1,rand_shuff_mean_RSA_1To1,rand_shuff_pval_RSA_1To1
numeric,0.02,0.01,0.19,-0.00,0.00
people,-0.02,0.01,0.66,0.01,0.67
nature,0.07,0.00,0.12,0.01,0.14
science,0.00,0.01,0.69,0.00,0.54
animals,0.50,0.10,0.55,0.08,0.51
